In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
# import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import pandas as pd
# import cudf as cd
# import dask_cudf as pd
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
from tqdm import tqdm
import glob
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
!pip install --upgrade polars

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.1/35.1 MB 46.2 MB/s eta 0:00:00
  Attempting uninstall: polars
    Found existing installation: polars 1.9.0
    Uninstalling polars-1.9.0:
      Successfully uninstalled polars-1.9.0


In [3]:
import polars
polars.__version__

'1.31.0'

In [4]:
seed = 1

In [5]:
train = pd.read_csv('/kaggle/input/solana-skill-sprint-memcoin-graduation/train.csv')
test  = pd.read_csv('/kaggle/input/solana-skill-sprint-memcoin-graduation/test_unlabeled.csv')
dune_token_df_1 = pd.read_csv('/kaggle/input/pump-fun-graduation-february-2025/dune_token_info.csv')
dune_token_df_2 = pd.read_csv('/kaggle/input/pump-fun-graduation-february-2025/dune_token_info_v2.csv')
onchain_df_1 = pd.read_csv('/kaggle/input/pump-fun-graduation-february-2025/token_info_onchain_divers.csv')
onchain_df_2 = pd.read_csv('/kaggle/input/pump-fun-graduation-february-2025/token_info_onchain_divers_v2.csv')
samp_sub = pd.read_csv('/kaggle/input/solana-skill-sprint-memcoin-graduation/sample_submission.csv')

train['is_train'] = 1
test['is_train'] = 0
combined_df = pd.concat([train, test], axis=0)

dune_token_df_combined = pd.concat([dune_token_df_1, dune_token_df_2], axis=0)

data_path = '/kaggle/input/pump-fun-graduation-february-2025'
chunk_pattern = os.path.join(data_path, 'chunk_*.csv')

chunk_files = sorted(glob.glob(chunk_pattern))

chunks = []
for f in tqdm(chunk_files, desc='Loading Chunks'):
    try:
        chunks.append(pd.read_csv(f, low_memory=False))
    except Exception as e:
        print(f'Error loading {os.path.basename(f)}: {str(e)}')

transactions = pd.concat(chunks, ignore_index=True)

<ipython-input-5-9b4bcdabe7bf>:5: DtypeWarning: Columns (1,2) have mixed types. Specify dtype option on import or set low_memory=False.
  onchain_df_1 = pd.read_csv('/kaggle/input/pump-fun-graduation-february-2025/token_info_onchain_divers.csv')
Loading Chunks: 100%|██████████| 41/41 [02:28<00:00,  3.62s/it]


In [6]:
# print(f'\nmissing values :')
# print(f'Train set : \n{train.isnull().sum()}')
# print('--------------')
# print(f'Test set : \n{test.isnull().sum()}')
# print('--------------')
# print(f'Dune set : \n{dune_token_df_combined.isnull().sum()}')
# print('--------------')
# print(f'Onchain_df_2 set : \n{onchain_df_2.isnull().sum()}')
# print('--------------')
# print(f'Transactions set : \n{transactions.isnull().sum()}')
# print('--------------')

In [7]:
# print(f'\nmissing values :')
# print(f'Train set : \n{train.describe()}')
# print('--------------')
# print(f'Test set : \n{test.describe()}')
# print('--------------')
# print(f'Dune set : \n{dune_token_df_combined.describe()}')
# print('--------------')
# print(f'Onchain_df_2 set : \n{onchain_df_2.describe()}')
# print('--------------')
# print(f'Transactions set : \n{transactions.describe()}')
# print('--------------')

In [8]:
# import matplotlib.pyplot as plt
# import seaborn as sns 

# sns.countplot(x='has_graduated', data=train)
# plt.title('Distribution of target label(train)')
# plt.xlabel('has_graduated')
# plt.ylabel('count')

In [9]:
# import warnings
# warnings.filterwarnings('ignore')
# plt.figure(figsize=(10, 6))
# sns.histplot(data=combined_df[combined_df['is_train']==1], x='slot_min', label='Train', alpha=0.5, kde=True)
# sns.histplot(data=combined_df[combined_df['is_train']==0], x='slot_min', label='Test', alpha=0.5, kde=True)
# plt.title('Distribution of slot min in train vs test')
# plt.xlabel('slot_min')
# plt.ylabel('count')
# plt.legend()

In [10]:
# plt.figure(figsize=(12, 6))
# pd.to_datetime(transactions['block_time']).dt.hour.hist(bins=24)
# plt.title('Transaction Activity by hour of day')
# plt.xlabel('Hour of day')
# plt.ylabel('Number of transactions')

In [11]:
transactions_df = pd.merge(
    transactions,
    combined_df[['mint', 'slot_min']],
    left_on='base_coin',
    right_on='mint',
    how='left'
)
transactions_df = transactions_df[transactions_df['slot'] <= transactions_df['slot_min'] + 100]

In [12]:
del transactions

In [13]:
# import polars as pl
# import numpy as np
# from datetime import datetime
# import concurrent.futures

# def fast_polars_process(transactions_df, n_jobs=4):
#     """Optimized version of mint processing using Polars operations"""
#     print(f"Starting Polars transaction processing at {datetime.now()}")
    
#     # Convert pandas dataframe to polars if needed
#     if not isinstance(transactions_df, pl.DataFrame):
#         transactions_df = pl.from_pandas(transactions_df)
    
#     # Ensure datetime conversion
#     if transactions_df["block_time"].dtype != pl.Datetime:
#         transactions_df = transactions_df.with_columns(
#             pl.col("block_time").str.to_datetime()
#         )
    
#     print("Calculating aggregated features...")
    
#     # Basic counts (vectorized)
#     counts = (
#         transactions_df
#         .group_by("base_coin")
#         .agg(
#             pl.count().alias("total_txs"),
#             pl.n_unique("tx_idx").alias("unique_tx_indices"),
#             pl.n_unique("signing_wallet").alias("unique_wallets")
#         )
#     )
    
#     # Direction-based metrics
#     direction_counts = (
#         transactions_df
#         .group_by(["base_coin", "direction"])
#         .agg(pl.count().alias("count"))
#         .pivot(index="base_coin", columns="direction", values="count")
#         .fill_null(0)
#     )
    
#     # Ensure buy and sell columns exist
#     if "buy" not in direction_counts.columns:
#         direction_counts = direction_counts.with_columns(pl.lit(0).alias("buy"))
#     if "sell" not in direction_counts.columns:
#         direction_counts = direction_counts.with_columns(pl.lit(0).alias("sell"))
    
#     # Rename columns
#     direction_counts = direction_counts.rename({"buy": "buy_count", "sell": "sell_count"})
    
#     # Wallet counts by direction
#     buy_wallets = (
#         transactions_df
#         .filter(pl.col("direction") == "buy")
#         .group_by("base_coin")
#         .agg(pl.n_unique("signing_wallet").alias("unique_buy_wallets"))
#     )
    
#     sell_wallets = (
#         transactions_df
#         .filter(pl.col("direction") == "sell")
#         .group_by("base_coin")
#         .agg(pl.n_unique("signing_wallet").alias("unique_sell_wallets"))
#     )
    
#     # Calculate base and quote statistics
#     base_stats = (
#         transactions_df
#         .group_by("base_coin")
#         .agg(
#             pl.col("base_coin_amount").sum().alias("base_sum"),
#             pl.col("base_coin_amount").mean().alias("base_mean"),
#             pl.col("base_coin_amount").max().alias("base_max"),
#             pl.col("base_coin_amount").min().alias("base_min"),
#             pl.col("base_coin_amount").std().alias("base_std")
#         )
#     )
    
#     quote_stats = (
#         transactions_df
#         .group_by("base_coin")
#         .agg(
#             pl.col("quote_coin_amount").sum().alias("quote_sum"),
#             pl.col("quote_coin_amount").mean().alias("quote_mean"),
#             pl.col("quote_coin_amount").max().alias("quote_max"),
#             pl.col("quote_coin_amount").min().alias("quote_min"),
#             pl.col("quote_coin_amount").std().alias("quote_std")
#         )
#     )
    
#     # Time-based features
#     time_stats = (
#         transactions_df
#         .group_by("base_coin")
#         .agg(
#             pl.col("block_time").min().alias("first_time"),
#             pl.col("block_time").max().alias("last_time")
#         )
#         .with_columns(
#             (pl.col("last_time") - pl.col("first_time")).dt.total_seconds().alias("activity_duration_sec")
#         )
#     )
#     # Transaction velocity calculation (NEW FEATURE)
#     time_stats = time_stats.join(counts.select("base_coin", "total_txs"), on="base_coin").with_columns(
#         (pl.col("total_txs") / (pl.col("activity_duration_sec") + 1e-6)).alias("tx_per_sec")
#     )
    
#     # Slot features
#     slot_stats = (
#         transactions_df
#         .group_by("base_coin")
#         .agg(
#             pl.col("slot").min().alias("first_slot"),
#             pl.col("slot").max().alias("last_slot")
#         )
#         .with_columns(
#             (pl.col("last_slot") - pl.col("first_slot")).alias("slot_span")
#         )
#     )
    
#     # Early transactions calculation (within first 100 slots)
#     early_txs = transactions_df.group_by("base_coin").agg(
#         pl.col("tx_idx")
#         .filter(pl.col("slot") <= pl.col("slot").min() + 100)
#         .count()
#         .alias("early_txs")
#     )
    
#     # Balance features - get first and last records efficiently
#     sorted_df = transactions_df.sort(["base_coin", "slot"])
    
#     # First values
#     first_values = (
#         sorted_df
#         .group_by("base_coin")
#         .agg(
#             pl.col("virtual_token_balance_after").first().alias("first_balance"),
#             pl.col("virtual_sol_balance_after").first().alias("first_sol")
#         )
#     )
    
#     # Last values
#     last_values = (
#         sorted_df
#         .group_by("base_coin")
#         .agg(
#             pl.col("virtual_token_balance_after").last().alias("last_balance"),
#             pl.col("virtual_sol_balance_after").last().alias("last_sol")
#         )
#     )
    
#     # Combine balance features
#     balance_features = first_values.join(last_values, on="base_coin").with_columns(
#         (pl.col("last_sol") - pl.col("first_sol")).alias("sol_change")
#     )
    
#     # Balance volatility
#     balance_vol = (
#         transactions_df
#         .group_by("base_coin")
#         .agg(
#             pl.col("virtual_token_balance_after").std().alias("balance_volatility"),
#             pl.col("virtual_token_balance_after").max().alias("max_balance"),
#             pl.col("virtual_token_balance_after").min().alias("min_balance")
#         )
#     )
    
#     # Gas features
#     gas_stats = (
#         transactions_df
#         .group_by("base_coin")
#         .agg(
#             pl.col("provided_gas_fee").sum().alias("total_gas_fees"),
#             pl.col("provided_gas_fee").mean().alias("avg_gas_fee"),
#             pl.col("provided_gas_fee").std().alias("std_gas_fee"),
#             pl.col("provided_gas_fee").max().alias("max_gas_fee"),
#             pl.col("provided_gas_fee").min().alias("min_gas_fee"),
#             pl.col("provided_gas_limit").mean().alias("avg_gas_limit"),
#             pl.col("consumed_gas").mean().alias("avg_consumed_gas"),
#             pl.col("consumed_gas").sum().alias("total_consumed_gas"),
#             pl.col("fee").sum().alias("total_fee"),
#             (pl.col("provided_gas_limit") - pl.col("consumed_gas")).mean().alias("limit_utilization")
#         )
#     )

#     liq_stats = (
#     transactions_df
#       .group_by("base_coin")
#       .agg([
#         # average SOL‐balance after each transaction (pool depth proxy)
#         pl.col("virtual_sol_balance_after")
#           .mean()
#           .alias("liq_virtual_sol_balance_after_mean"),
#         # total quote‐volume (another liquidity proxy)
#         pl.col("quote_coin_amount")
#           .sum()
#           .alias("liq_quote_coin_amount"),
#       ])
#     )
#     # Gas metrics with quote
#     gas_metrics = gas_stats.join(quote_stats.select(["base_coin", "quote_sum"]), on="base_coin").with_columns([
#         (pl.col("total_fee") / (pl.col("quote_sum") + 1e-6)).alias("fee_per_sol"),
#         pl.col("avg_consumed_gas").alias("gas_per_tx"),
#         (pl.col("quote_sum") / (pl.col("total_consumed_gas") + 1e-6)).alias("gas_efficiency")
#     ])
    
#     # Wallet concentration (Gini and top wallet share) - using chunks to avoid memory issues
#     def calculate_concentration_metrics(mint):
#         mint_df = transactions_df.filter(pl.col("base_coin") == mint)
#         wallet_volumes = mint_df.group_by("signing_wallet").agg(pl.col("base_coin_amount").sum())
        
#         if wallet_volumes.height <= 1:
#             return {"base_coin": mint, "gini_coeff": 0.0, "top5_wallet_share": 0.0}
        
#         sorted_values = wallet_volumes.sort("base_coin_amount").to_pandas()["base_coin_amount"].values
#         n = len(sorted_values)
        
#         if n > 1:
#             gini_coeff = np.sum((2 * np.arange(1, n+1) - n - 1) * sorted_values) / (n * np.sum(sorted_values) + 1e-6)
#         else:
#             gini_coeff = 0.0
        
#         total_volume = sorted_values.sum()
#         if total_volume > 0:
#             top_k = min(5, n)
#             top_k_volume = np.sum(sorted_values[-top_k:])
#             top5_wallet_share = top_k_volume / total_volume
#         else:
#             top5_wallet_share = 0.0
            
#         return {"base_coin": mint, "gini_coeff": gini_coeff, "top5_wallet_share": top5_wallet_share}
    
#     print("Calculating wallet concentration metrics...")
#     unique_mints = counts["base_coin"].to_list()
    
#     # Use ThreadPoolExecutor for parallel processing
#     with concurrent.futures.ThreadPoolExecutor(max_workers=n_jobs) as executor:
#         concentration_results = list(executor.map(calculate_concentration_metrics, unique_mints))
    
#     concentration_df = pl.DataFrame(concentration_results)
    
#     # Hour distribution entropy
#     def calculate_hour_entropy(mint):
#         mint_df = transactions_df.filter(pl.col("base_coin") == mint)
#         hours = mint_df["block_time"].dt.hour()
#         hour_counts = hours.value_counts().sort("count", descending=True)
        
#         if hour_counts.height > 0:
#             total = hour_counts["count"].sum()
#             probs = hour_counts["count"] / total
#             non_zero_probs = probs.filter(probs > 0)
            
#             if non_zero_probs.len() > 0:
#                 entropy = -np.sum(non_zero_probs.to_numpy() * np.log2(non_zero_probs.to_numpy()))
#                 return {"base_coin": mint, "hour_entropy": entropy}
        
#         return {"base_coin": mint, "hour_entropy": 0.0}
    
#     print("Calculating hour entropy...")
#     with concurrent.futures.ThreadPoolExecutor(max_workers=n_jobs) as executor:
#         hour_entropy_results = list(executor.map(calculate_hour_entropy, unique_mints))
    
#     hour_entropy_df = pl.DataFrame(hour_entropy_results)
    
#     # Flow imbalance calculation
#     flow_imbalance = (
#         transactions_df
#         .group_by("base_coin")
#         .agg([
#             pl.col("quote_coin_amount").filter(pl.col("direction") == "buy").sum().alias("buy_quote_amount"),
#             pl.col("quote_coin_amount").filter(pl.col("direction") == "sell").sum().alias("sell_quote_amount")
#         ])
#         .with_columns([
#             (pl.col("buy_quote_amount") + pl.col("sell_quote_amount")).alias("total_quote_amount"),
#             ((pl.col("buy_quote_amount") - pl.col("sell_quote_amount")) / 
#              (pl.col("buy_quote_amount") + pl.col("sell_quote_amount") + 1e-6)).alias("flow_imbalance")
#         ])
#         .select(["base_coin", "flow_imbalance"])
#     )
    
#     # Price and volatility calculations
#     def calculate_price_metrics(mint):
#         mint_df = transactions_df.filter(pl.col("base_coin") == mint)
        
#         # Calculate price
#         mint_df = mint_df.with_columns(
#             (pl.col("quote_coin_amount") / pl.col("base_coin_amount")
#              .replace(0, None))
#             .alias("price")
#         )
        
#         # Replace infinite values with nulls
#         mint_df = mint_df.with_columns(
#             pl.when(pl.col("price").is_infinite())
#             .then(None)
#             .otherwise(pl.col("price"))
#             .alias("price")
#         )
        
#         avg_price = mint_df["price"].mean()
        
#         if mint_df.height >= 5:
#             # For rolling window calculations, we need to sort and convert to pandas temporarily
#             # This is a performance bottleneck, but necessary for the rolling calculation
#             sorted_df = mint_df.sort("slot").to_pandas()
#             sorted_df['rolling_price_std'] = sorted_df['price'].rolling(window=5, min_periods=1).std().fillna(0)
#             price_vol_mean = sorted_df['rolling_price_std'].mean()
#             price_vol_max = sorted_df['rolling_price_std'].max()
#         else:
#             price_vol_mean = mint_df["price"].std() if mint_df.height > 1 else 0
#             price_vol_max = price_vol_mean
        
#         price_min = mint_df["price"].min()
#         price_max = mint_df["price"].max()
#         price_mean = mint_df["price"].mean()
        
#         price_slippage = (price_max - price_min) / (price_mean + 1e-6) if price_mean is not None else 0
        
#         return {
#             "base_coin": mint,
#             "avg_price": float(avg_price) if avg_price is not None else 0.0,
#             "price_vol_mean": float(price_vol_mean) if price_vol_mean is not None else 0.0,
#             "price_vol_max": float(price_vol_max) if price_vol_max is not None else 0.0,
#             "price_slippage": float(price_slippage)
#         }
    
#     print("Calculating price and volatility metrics...")
#     with concurrent.futures.ThreadPoolExecutor(max_workers=n_jobs) as executor:
#         price_results = list(executor.map(calculate_price_metrics, unique_mints))
    
#     price_df = pl.DataFrame(price_results)
    
#     # Time-weighted features
#     def calculate_tw_features(mint):
#         mint_df = transactions_df.filter(pl.col("base_coin") == mint)
#         max_slot = mint_df["slot"].max()
        
#         # Calculate time weights using numpy instead of pl.exp
#         mint_df = mint_df.with_columns(
#             pl.col("slot").map_elements(lambda s: np.exp(-0.1 * (max_slot - s))).alias("time_weight")
#         )
        
#         tw_base = (mint_df["base_coin_amount"] * mint_df["time_weight"]).sum()
#         tw_quote = (mint_df["quote_coin_amount"] * mint_df["time_weight"]).sum()
        
#         return {
#             "base_coin": mint,
#             "tw_base_coin_amount": float(tw_base),
#             "tw_quote_coin_amount": float(tw_quote)
#         }
    
#     print("Calculating time-weighted features...")
#     with concurrent.futures.ThreadPoolExecutor(max_workers=n_jobs) as executor:
#         tw_results = list(executor.map(calculate_tw_features, unique_mints))
    
#     tw_df = pl.DataFrame(tw_results)
    
#     # Now merge all feature DataFrames
#     print("Merging all features...")
    
#     # Start with counts as the base
#     result = counts
    
#     # Add buy ratio
#     result = result.join(direction_counts, on="base_coin", how="left")
#     result = result.with_columns(
#         (pl.col("buy_count") / pl.col("total_txs")).alias("buy_ratio")
#     )
    
#     # Join remaining dataframes
#     dfs_to_join = [
#         buy_wallets, sell_wallets, base_stats, quote_stats, 
#         time_stats.select(["base_coin", "activity_duration_sec", "first_time", "last_time", "tx_per_sec"]), 
#         slot_stats, early_txs, balance_features, balance_vol, 
#         gas_metrics, concentration_df, hour_entropy_df, flow_imbalance, price_df, tw_df, liq_stats
#     ]
    
#     for df in dfs_to_join:
#         result = result.join(df, on="base_coin", how="left")
    
#     # Rename base_coin to mint for consistency with original
#     result = result.rename({"base_coin": "mint"})
    
#     # Fill null values
#     result = result.fill_null(0)
    
#     print(f"Polars processing complete with {result.height} rows and {len(result.columns)} columns")
    
#     # Convert back to pandas if needed
#     return result.to_pandas()

# start_time = datetime.now()
# mint_transactions = fast_polars_process(transactions_df.iloc[:100000], n_jobs=4)
# end_time = datetime.now()
# elapsed_time = end_time - start_time
# print(f'Time it took to process: {elapsed_time}')

In [14]:
# import polars as pl
# import numpy as np
# from datetime import datetime
# import concurrent.futures
# import time
# import cudf
# import cupy as cp

# def fast_polars_process(transactions_df, n_jobs=4):
#     """Fully optimized version using Polars LazyFrames and GPU acceleration"""
#     total_start_time = time.time()
#     print(f"Starting GPU-accelerated Polars transaction processing at {datetime.now()}")
    
#     # Convert to LazyFrame
#     convert_start = time.time()
#     if not isinstance(transactions_df, pl.LazyFrame):
#         if not isinstance(transactions_df, pl.DataFrame):
#             transactions_df = pl.from_pandas(transactions_df)
#         lf = transactions_df.lazy()
#     else:
#         lf = transactions_df
#     convert_end = time.time()
#     print(f"Time to convert to LazyFrame: {convert_end - convert_start:.4f} seconds")
    
#     # Ensure datetime conversion
#     datetime_start = time.time()
#     lf = lf.with_columns(
#         pl.col("block_time").str.to_datetime(strict=False)
#     )
#     datetime_end = time.time()
#     print(f"Time to convert block_time to datetime: {datetime_end - datetime_start:.4f} seconds")
    
#     print("Calculating aggregated features...")
    
#     # Basic counts
#     counts_start = time.time()
#     counts = (
#         lf.group_by("base_coin")
#         .agg(
#             pl.len().alias("total_txs"),
#             pl.n_unique("tx_idx").alias("unique_tx_indices"),
#             pl.n_unique("signing_wallet").alias("unique_wallets")
#         )
#     )
#     counts_end = time.time()
#     print(f"Time to compute basic counts: {counts_end - counts_start:.4f} seconds")
    
#     # Direction-based metrics
#     dir_start = time.time()
#     direction_counts = (
#         lf.group_by("base_coin")
#         .agg(
#             pl.when(pl.col("direction") == "buy")
#               .then(1)
#               .otherwise(0)
#               .sum()
#               .alias("buy_count"),
#             pl.when(pl.col("direction") == "sell")
#               .then(1)
#               .otherwise(0)
#               .sum()
#               .alias("sell_count")
#         )
#     )
#     dir_end = time.time()
#     print(f"Time to compute direction-based metrics: {dir_end - dir_start:.4f} seconds")
    
#     # Wallet counts - collect with GPU
#     wallet_start = time.time()
#     buy_wallets = (
#         lf.filter(pl.col("direction") == "buy")
#         .group_by("base_coin")
#         .agg(pl.n_unique("signing_wallet").alias("unique_buy_wallets"))
#     )
    
#     sell_wallets = (
#         lf.filter(pl.col("direction") == "sell")
#         .group_by("base_coin")
#         .agg(pl.n_unique("signing_wallet").alias("unique_sell_wallets"))
#     )
#     wallet_end = time.time()
#     print(f"Time to compute wallet counts: {wallet_end - wallet_start:.4f} seconds")
    
#     # Base/quote statistics - collect with GPU
#     stats_start = time.time()
#     base_stats = (
#         lf.group_by("base_coin")
#         .agg(
#             pl.col("base_coin_amount").sum().alias("base_sum"),
#             pl.col("base_coin_amount").mean().alias("base_mean"),
#             pl.col("base_coin_amount").max().alias("base_max"),
#             pl.col("base_coin_amount").min().alias("base_min"),
#             pl.col("base_coin_amount").std().alias("base_std")
#         )
#     )
    
#     quote_stats = (
#         lf.group_by("base_coin")
#         .agg(
#             pl.col("quote_coin_amount").sum().alias("quote_sum"),
#             pl.col("quote_coin_amount").mean().alias("quote_mean"),
#             pl.col("quote_coin_amount").max().alias("quote_max"),
#             pl.col("quote_coin_amount").min().alias("quote_min"),
#             pl.col("quote_coin_amount").std().alias("quote_std")
#         )
#     )
#     stats_end = time.time()
#     print(f"Time to compute base/quote statistics: {stats_end - stats_start:.4f} seconds")
    
#     # Time-based features - collect with GPU
#     time_feat_start = time.time()
#     time_stats = (
#         lf.group_by("base_coin")
#         .agg(
#             pl.col("block_time").min().alias("first_time"),
#             pl.col("block_time").max().alias("last_time")
#         )
#         .with_columns(
#             (pl.col("last_time") - pl.col("first_time"))
#             .dt.total_seconds()
#             .alias("activity_duration_sec")
#         )
#         .join(counts.select("base_coin", "total_txs"), on="base_coin")
#         .with_columns(
#             (pl.col("total_txs") / (pl.col("activity_duration_sec") + 1e-6))
#             .alias("tx_per_sec")
#         )
#     )
#     time_feat_end = time.time()
#     print(f"Time to compute time-based features: {time_feat_end - time_feat_start:.4f} seconds")
    
#     # Slot features - collect with GPU
#     slot_start = time.time()
#     slot_stats = (
#         lf.group_by("base_coin")
#         .agg(
#             pl.col("slot").min().alias("first_slot"),
#             pl.col("slot").max().alias("last_slot")
#         )
#         .with_columns(
#             (pl.col("last_slot") - pl.col("first_slot")).alias("slot_span")
#         )
#     )
    
#     # Early transactions - collect with GPU
#     early_txs = (
#         lf.group_by("base_coin")
#         .agg(
#             pl.col("tx_idx")
#             .filter(pl.col("slot") <= pl.col("slot").min() + 100)
#             .count()
#             .alias("early_txs")
#         )
#     )
#     slot_end = time.time()
#     print(f"Time to compute slot features: {slot_end - slot_start:.4f} seconds")
    
#     # Balance features - collect with GPU
#     balance_start = time.time()
#     sorted_lf = lf.sort(["base_coin", "slot"])
#     first_values = (
#         sorted_lf.group_by("base_coin")
#         .agg(
#             pl.col("virtual_token_balance_after").first().alias("first_balance"),
#             pl.col("virtual_sol_balance_after").first().alias("first_sol")
#         )
#     )
#     last_values = (
#         sorted_lf.group_by("base_coin")
#         .agg(
#             pl.col("virtual_token_balance_after").last().alias("last_balance"),
#             pl.col("virtual_sol_balance_after").last().alias("last_sol")
#         )
#     )
#     balance_features = (
#         first_values.join(last_values, on="base_coin")
#         .with_columns(
#             (pl.col("last_sol") - pl.col("first_sol")).alias("sol_change")
#         )
#     )
    
#     # Balance volatility - collect with GPU
#     balance_vol = (
#         lf.group_by("base_coin")
#         .agg(
#             pl.col("virtual_token_balance_after").std().alias("balance_volatility"),
#             pl.col("virtual_token_balance_after").max().alias("max_balance"),
#             pl.col("virtual_token_balance_after").min().alias("min_balance")
#         )
#     )
#     balance_end = time.time()
#     print(f"Time to compute balance features: {balance_end - balance_start:.4f} seconds")
    
#     # Gas features - collect with GPU
#     gas_start = time.time()
#     gas_stats = (
#         lf.group_by("base_coin")
#         .agg(
#             pl.col("provided_gas_fee").sum().alias("total_gas_fees"),
#             pl.col("provided_gas_fee").mean().alias("avg_gas_fee"),
#             pl.col("provided_gas_fee").std().alias("std_gas_fee"),
#             pl.col("provided_gas_fee").max().alias("max_gas_fee"),
#             pl.col("provided_gas_fee").min().alias("min_gas_fee"),
#             pl.col("provided_gas_limit").mean().alias("avg_gas_limit"),
#             pl.col("consumed_gas").mean().alias("avg_consumed_gas"),
#             pl.col("consumed_gas").sum().alias("total_consumed_gas"),
#             pl.col("fee").sum().alias("total_fee"),
#             (pl.col("provided_gas_limit") - pl.col("consumed_gas")).mean().alias("limit_utilization")
#         )
#     )
    
#     # Liquidity stats - collect with GPU
#     liq_stats = (
#         lf.group_by("base_coin")
#         .agg(
#             pl.col("virtual_sol_balance_after").mean().alias("liq_virtual_sol_balance_after_mean"),
#             pl.col("quote_coin_amount").sum().alias("liq_quote_coin_amount")
#         )
#     )
    
#     # Gas metrics - collect with GPU
#     gas_metrics = (
#         gas_stats.join(quote_stats.select(["base_coin", "quote_sum"]), on="base_coin")
#         .with_columns([
#             (pl.col("total_fee") / (pl.col("quote_sum") + 1e-6)).alias("fee_per_sol"),
#             pl.col("avg_consumed_gas").alias("gas_per_tx"),
#             (pl.col("quote_sum") / (pl.col("total_consumed_gas") + 1e-6)).alias("gas_efficiency")
#         ])
#     )
#     gas_end = time.time()
#     print(f"Time to compute gas and liquidity features: {gas_end - gas_start:.4f} seconds")
    
#     # Flow imbalance - collect with GPU
#     flow_start = time.time()
#     flow_imbalance = (
#         lf.group_by("base_coin")
#         .agg(
#             pl.col("quote_coin_amount").filter(pl.col("direction") == "buy").sum().alias("buy_quote_amount"),
#             pl.col("quote_coin_amount").filter(pl.col("direction") == "sell").sum().alias("sell_quote_amount")
#         )
#         .with_columns(
#             ((pl.col("buy_quote_amount") - pl.col("sell_quote_amount")) / 
#              (pl.col("buy_quote_amount") + pl.col("sell_quote_amount") + 1e-6)).alias("flow_imbalance")
#         )
#         .select(["base_coin", "flow_imbalance"])
#     )
#     flow_end = time.time()
#     print(f"Time to compute flow imbalance features: {flow_end - flow_start:.4f} seconds")
    
#     # Collect necessary data for custom functions
#     print("Collecting data for custom functions...")
#     collect_start = time.time()
#     unique_mints = counts.collect(engine='gpu')["base_coin"].to_list()
#     collect_end = time.time()
#     print(f"Time to collect unique mints: {collect_end - collect_start:.4f} seconds")
    
#     # Wallet concentration metrics - GPU-optimized version
#     wallet_conc_start = time.time()
#     print("Calculating wallet concentration metrics using GPU...")

#     # Pre-calculate wallet volumes for all mints at once
#     wallet_volumes = (
#         lf.group_by(["base_coin", "signing_wallet"])
#         .agg(pl.col("base_coin_amount").sum().alias("volume"))
#         .collect(engine='gpu')
#     )

#     # Convert to CUDF for faster GPU-based calculations
#     try:
#         cudf_wallet_volumes = cudf.DataFrame.from_pandas(wallet_volumes.to_pandas())
        
#         # Function to calculate Gini and top5 share for a single mint using CUDA
#         def calculate_concentration_metrics_gpu(mint):
#             mint_volumes = cudf_wallet_volumes[cudf_wallet_volumes['base_coin'] == mint]
            
#             if len(mint_volumes) <= 1:
#                 return {"base_coin": mint, "gini_coeff": 0.0, "top5_wallet_share": 0.0}
            
#             sorted_values = mint_volumes.sort_values('volume')['volume'].values
#             sorted_values_cp = cp.array(sorted_values)
#             n = len(sorted_values)
            
#             # Gini coefficient calculation using CuPy
#             cumulative_values = cp.cumsum(sorted_values_cp)
#             total = cumulative_values[-1]
#             if total <= 0:
#                 return {"base_coin": mint, "gini_coeff": 0.0, "top5_wallet_share": 0.0}
            
#             # Faster Gini calculation using cumulative sum
#             gini_coeff = (n + 1 - 2 * cp.sum(cumulative_values) / total) / n
            
#             # Top wallet share
#             top_k = min(5, n)
#             top5_wallet_share = cp.sum(sorted_values_cp[-top_k:]) / total
            
#             return {"base_coin": mint, "gini_coeff": float(gini_coeff), "top5_wallet_share": float(top5_wallet_share)}
        
#         concentration_results = []
#         for mint in unique_mints:
#             concentration_results.append(calculate_concentration_metrics_gpu(mint))
        
#     except (ImportError, ModuleNotFoundError):
#         print("CUDF not available, falling back to CPU calculation")
#         # Function to calculate Gini and top5 share for a single mint using pre-calculated data
#         def calculate_concentration_metrics_optimized(mint):
#             mint_volumes = wallet_volumes.filter(pl.col("base_coin") == mint)
            
#             if mint_volumes.height <= 1:
#                 return {"base_coin": mint, "gini_coeff": 0.0, "top5_wallet_share": 0.0}
            
#             sorted_values = mint_volumes.sort("volume")["volume"].to_numpy()
#             n = len(sorted_values)
            
#             # Gini coefficient calculation
#             cumulative_values = np.cumsum(sorted_values)
#             total = cumulative_values[-1]
#             if total <= 0:
#                 return {"base_coin": mint, "gini_coeff": 0.0, "top5_wallet_share": 0.0}
            
#             # Faster Gini calculation using cumulative sum
#             gini_coeff = (n + 1 - 2 * np.sum(cumulative_values) / total) / n
            
#             # Top wallet share
#             top_k = min(5, n)
#             top5_wallet_share = np.sum(sorted_values[-top_k:]) / total
            
#             return {"base_coin": mint, "gini_coeff": float(gini_coeff), "top5_wallet_share": float(top5_wallet_share)}

#         # Process in parallel using the pre-calculated data
#         with concurrent.futures.ThreadPoolExecutor(max_workers=n_jobs) as executor:
#             concentration_results = list(executor.map(calculate_concentration_metrics_optimized, unique_mints))
            
#     concentration_df = pl.DataFrame(concentration_results)

#     wallet_conc_end = time.time()
#     print(f"Time to compute wallet concentration metrics: {wallet_conc_end - wallet_conc_start:.4f} seconds")

#     # Hour entropy - GPU-optimized version
#     hour_start = time.time()
#     print("Calculating hour entropy using GPU...")

#     # Calculate hours for all records at once
#     hours_df = (
#         lf.select([
#             pl.col("base_coin"),
#             pl.col("block_time").dt.hour().alias("hour")
#         ])
#         .collect(engine='gpu')
#     )

#     # Calculate hour counts for all mints at once
#     hour_counts = (
#         hours_df.group_by(["base_coin", "hour"])
#         .agg(pl.len().alias("count"))
#     )

#     try:
#         # Convert to CUDF for faster GPU-based calculations
#         cudf_hour_counts = cudf.DataFrame.from_pandas(hour_counts.to_pandas())
#         cudf_hour_counts_grouped = cudf_hour_counts.groupby('base_coin')
        
#         mint_entropy = []
#         for mint in unique_mints:
#             try:
#                 mint_hours = cudf_hour_counts[cudf_hour_counts['base_coin'] == mint]
#                 if len(mint_hours) == 0:
#                     mint_entropy.append({"base_coin": mint, "hour_entropy": 0.0})
#                     continue
                
#                 total = mint_hours['count'].sum()
#                 if total == 0:
#                     mint_entropy.append({"base_coin": mint, "hour_entropy": 0.0})
#                     continue
                
#                 probs = mint_hours['count'] / total
#                 probs_cp = cp.array(probs.values)
#                 non_zero_probs = probs_cp[probs_cp > 0]
                
#                 if len(non_zero_probs) == 0:
#                     mint_entropy.append({"base_coin": mint, "hour_entropy": 0.0})
#                     continue
                
#                 entropy = -cp.sum(non_zero_probs * cp.log2(non_zero_probs))
#                 mint_entropy.append({"base_coin": mint, "hour_entropy": float(entropy)})
#             except Exception as e:
#                 print(f"Error processing mint {mint} for hour entropy: {e}")
#                 mint_entropy.append({"base_coin": mint, "hour_entropy": 0.0})
                
#     except (ImportError, ModuleNotFoundError):
#         print("CUDF not available, falling back to CPU calculation")
#         # Calculate entropy for each mint
#         mint_entropy = []
#         for mint in unique_mints:
#             mint_hours = hour_counts.filter(pl.col("base_coin") == mint)
#             if mint_hours.height == 0:
#                 mint_entropy.append({"base_coin": mint, "hour_entropy": 0.0})
#                 continue
            
#             total = mint_hours["count"].sum()
#             if total == 0:
#                 mint_entropy.append({"base_coin": mint, "hour_entropy": 0.0})
#                 continue
            
#             probs = mint_hours["count"] / total
#             non_zero_probs = probs.filter(probs > 0).to_numpy()
            
#             if len(non_zero_probs) == 0:
#                 mint_entropy.append({"base_coin": mint, "hour_entropy": 0.0})
#                 continue
            
#             entropy = -np.sum(non_zero_probs * np.log2(non_zero_probs))
#             mint_entropy.append({"base_coin": mint, "hour_entropy": float(entropy)})
        
#     hour_entropy_df = pl.DataFrame(mint_entropy)
#     hour_end = time.time()
#     print(f"Time to compute hour entropy: {hour_end - hour_start:.4f} seconds")

#     # Price and volatility - GPU-optimized version
#     price_start = time.time()
#     print("Calculating price and volatility metrics using GPU...")

#     # Calculate prices for all records at once
#     price_df_all = (
#         lf.with_columns(
#             (pl.col("quote_coin_amount") / pl.col("base_coin_amount"))
#             .replace(0, None)
#             .alias("price")
#         )
#         .select(["base_coin", "slot", "price"])
#         .collect(engine='gpu')
#         .filter(
#             (pl.col("price").is_not_nan()) & 
#             (pl.col("price").is_not_null()) & 
#             (pl.col("price").is_finite())
#         )
#         .sort(["base_coin", "slot"])
#     )

#     try:
#         # Convert to CUDF for faster GPU-based calculations
#         cudf_price_df = cudf.DataFrame.from_pandas(price_df_all.to_pandas())
        
#         # Calculate price metrics for each mint
#         price_metrics = []
#         for mint in unique_mints:
#             try:
#                 mint_prices = cudf_price_df[cudf_price_df['base_coin'] == mint]
                
#                 if len(mint_prices) == 0:
#                     price_metrics.append({
#                         "base_coin": mint,
#                         "avg_price": 0.0,
#                         "price_vol_mean": 0.0,
#                         "price_vol_max": 0.0,
#                         "price_slippage": 0.0
#                     })
#                     continue
                
#                 prices_cp = cp.array(mint_prices['price'].values)
#                 avg_price = float(cp.mean(prices_cp))
#                 price_min = float(cp.min(prices_cp))
#                 price_max = float(cp.max(prices_cp))
                
#                 # Calculate rolling standard deviation with CuPy
#                 if len(prices_cp) >= 5:
#                     window_size = 5
#                     rolling_stds = []
#                     for i in range(len(prices_cp) - window_size + 1):
#                         window = prices_cp[i:i+window_size]
#                         rolling_stds.append(float(cp.std(window)))
#                     if rolling_stds:
#                         price_vol_mean = float(cp.mean(cp.array(rolling_stds)))
#                         price_vol_max = float(cp.max(cp.array(rolling_stds)))
#                     else:
#                         price_vol_mean = float(cp.std(prices_cp))
#                         price_vol_max = price_vol_mean
#                 else:
#                     price_vol_mean = float(cp.std(prices_cp))
#                     price_vol_max = price_vol_mean
                
#                 price_slippage = (price_max - price_min) / (avg_price + 1e-6) if avg_price != 0 else 0.0
                
#                 price_metrics.append({
#                     "base_coin": mint,
#                     "avg_price": float(avg_price),
#                     "price_vol_mean": float(price_vol_mean),
#                     "price_vol_max": float(price_vol_max),
#                     "price_slippage": float(price_slippage)
#                 })
#             except Exception as e:
#                 print(f"Error processing mint {mint} for price metrics: {e}")
#                 price_metrics.append({
#                     "base_coin": mint,
#                     "avg_price": 0.0,
#                     "price_vol_mean": 0.0,
#                     "price_vol_max": 0.0,
#                     "price_slippage": 0.0
#                 })
                
#     except (ImportError, ModuleNotFoundError):
#         print("CUDF/CuPy not available, falling back to CPU calculation")
#         # Calculate price metrics for each mint
#         price_metrics = []
#         for mint in unique_mints:
#             mint_prices = price_df_all.filter(pl.col("base_coin") == mint)
            
#             if mint_prices.height == 0:
#                 price_metrics.append({
#                     "base_coin": mint,
#                     "avg_price": 0.0,
#                     "price_vol_mean": 0.0,
#                     "price_vol_max": 0.0,
#                     "price_slippage": 0.0
#                 })
#                 continue
            
#             prices = mint_prices["price"].to_numpy()
#             avg_price = np.mean(prices)
#             price_min = np.min(prices)
#             price_max = np.max(prices)
            
#             # Calculate rolling standard deviation without pandas
#             if len(prices) >= 5:
#                 # Calculate rolling window standard deviations with numpy
#                 window_size = 5
#                 rolling_stds = []
#                 for i in range(len(prices) - window_size + 1):
#                     window = prices[i:i+window_size]
#                     rolling_stds.append(np.std(window))
#                 if rolling_stds:
#                     price_vol_mean = np.mean(rolling_stds)
#                     price_vol_max = np.max(rolling_stds)
#                 else:
#                     price_vol_mean = np.std(prices)
#                     price_vol_max = price_vol_mean
#             else:
#                 price_vol_mean = np.std(prices)
#                 price_vol_max = price_vol_mean
            
#             price_slippage = (price_max - price_min) / (avg_price + 1e-6) if avg_price != 0 else 0.0
            
#             price_metrics.append({
#                 "base_coin": mint,
#                 "avg_price": float(avg_price),
#                 "price_vol_mean": float(price_vol_mean),
#                 "price_vol_max": float(price_vol_max),
#                 "price_slippage": float(price_slippage)
#             })

#     price_df = pl.DataFrame(price_metrics)
#     price_end = time.time()
#     print(f"Time to compute price and volatility metrics: {price_end - price_start:.4f} seconds")

#     # Time-weighted features - GPU-optimized version
#     tw_start = time.time()
#     print("Calculating time-weighted features using GPU...")

#     # Get max slot for each base_coin in one operation
#     max_slots = (
#         lf.group_by("base_coin")
#         .agg(pl.col("slot").max().alias("max_slot"))
#         .collect(engine='gpu')
#     )

#     # Create expanded dataframe with time weights
#     tw_expanded = (
#         lf.select([
#             "base_coin", "slot", "base_coin_amount", "quote_coin_amount"
#         ])
#         .collect(engine='gpu')
#         .join(max_slots, on="base_coin", how="left")
#         .with_columns(
#             (0.1 * (pl.col("slot") - pl.col("max_slot")))
#             .exp()
#             .alias("time_weight")
#         )
#     )

#     try:
#         # Convert to CUDF for faster GPU-based calculations
#         cudf_tw_expanded = cudf.DataFrame.from_pandas(tw_expanded.to_pandas())
        
#         # Calculate time-weighted sums for all mints at once using GPU
#         tw_results_gpu = []
#         for mint in unique_mints:
#             mint_data = cudf_tw_expanded[cudf_tw_expanded['base_coin'] == mint]
#             if len(mint_data) == 0:
#                 tw_results_gpu.append({
#                     "base_coin": mint,
#                     "tw_base_coin_amount": 0.0,
#                     "tw_quote_coin_amount": 0.0
#                 })
#                 continue
            
#             tw_base = float((mint_data['base_coin_amount'] * mint_data['time_weight']).sum())
#             tw_quote = float((mint_data['quote_coin_amount'] * mint_data['time_weight']).sum())
            
#             tw_results_gpu.append({
#                 "base_coin": mint,
#                 "tw_base_coin_amount": tw_base if not np.isnan(tw_base) else 0.0,
#                 "tw_quote_coin_amount": tw_quote if not np.isnan(tw_quote) else 0.0
#             })
            
#         tw_df = pl.DataFrame(tw_results_gpu)
        
#     except (ImportError, ModuleNotFoundError):
#         print("CUDF not available, falling back to CPU calculation")
#         # Calculate time-weighted sums for all mints at once
#         tw_results = (
#             tw_expanded.group_by("base_coin")
#             .agg([
#                 (pl.col("base_coin_amount") * pl.col("time_weight")).sum().alias("tw_base_coin_amount"),
#                 (pl.col("quote_coin_amount") * pl.col("time_weight")).sum().alias("tw_quote_coin_amount")
#             ])
#             .with_columns([
#                 pl.col("tw_base_coin_amount").fill_null(0.0).cast(pl.Float64),
#                 pl.col("tw_quote_coin_amount").fill_null(0.0).cast(pl.Float64)
#             ])
#         )
#         tw_df = tw_results

#     tw_end = time.time()
#     print(f"Time to compute time-weighted features: {tw_end - tw_start:.4f} seconds")
    
#     # Collect all lazy results with GPU engine
#     print("Collecting lazy results with GPU engine...")
#     collect_start = time.time()
#     collected = {
#         "counts": counts.collect(engine='gpu'),
#         "direction_counts": direction_counts.collect(engine='gpu'),
#         "buy_wallets": buy_wallets.collect(engine='gpu'),
#         "sell_wallets": sell_wallets.collect(engine='gpu'),
#         "base_stats": base_stats.collect(engine='gpu'),
#         "quote_stats": quote_stats.collect(engine='gpu'),
#         "time_stats": time_stats.collect(engine='gpu'),
#         "slot_stats": slot_stats.collect(engine='gpu'),
#         "early_txs": early_txs.collect(engine='gpu'),
#         "balance_features": balance_features.collect(engine='gpu'),
#         "balance_vol": balance_vol.collect(engine='gpu'),
#         "gas_metrics": gas_metrics.collect(engine='gpu'),
#         "liq_stats": liq_stats.collect(engine='gpu'),
#         "flow_imbalance": flow_imbalance.collect(engine='gpu')
#     }
#     collect_end = time.time()
#     print(f"Time to collect all lazy results: {collect_end - collect_start:.4f} seconds")
    
#     # Merge all features
#     print("Merging all features...")
#     merge_start = time.time()
#     result = collected["counts"].join(collected["direction_counts"], on="base_coin", how="left")
#     result = result.with_columns(
#         (pl.col("buy_count") / pl.col("total_txs")).alias("buy_ratio")
#     )
    
#     dfs_to_join = [
#         collected["buy_wallets"],
#         collected["sell_wallets"],
#         collected["base_stats"],
#         collected["quote_stats"],
#         collected["time_stats"].select(["base_coin", "activity_duration_sec", "first_time", "last_time", "tx_per_sec"]),
#         collected["slot_stats"],
#         collected["early_txs"],
#         collected["balance_features"],
#         collected["balance_vol"],
#         collected["gas_metrics"],
#         concentration_df,
#         hour_entropy_df,
#         collected["flow_imbalance"],
#         price_df,
#         tw_df,
#         collected["liq_stats"]
#     ]
    
#     for df in dfs_to_join:
#         result = result.join(df, on="base_coin", how="left")
#     merge_end = time.time()
#     print(f"Time to join all features: {merge_end - merge_start:.4f} seconds")
    
#     # Final processing
#     final_start = time.time()
#     result = result.rename({"base_coin": "mint"}).fill_null(0)
#     final_end = time.time()
#     print(f"Time for final processing: {final_end - final_start:.4f} seconds")
    
#     total_end_time = time.time()
#     print(f"Total processing time: {total_end_time - total_start_time:.4f} seconds")
#     print(f"Polars processing complete with {result.height} rows and {len(result.columns)} columns")
    
#     # Summary of time profiling
#     print("\n=== TIMING SUMMARY ===")
#     timing_data = [
#         ("Convert to LazyFrame", convert_end - convert_start),
#         ("Datetime conversion", datetime_end - datetime_start),
#         ("Basic counts", counts_end - counts_start),
#         ("Direction-based metrics", dir_end - dir_start),
#         ("Wallet counts", wallet_end - wallet_start),
#         ("Base/quote statistics", stats_end - stats_start),
#         ("Time-based features", time_feat_end - time_feat_start),
#         ("Slot features", slot_end - slot_start),
#         ("Balance features", balance_end - balance_start),
#         ("Gas and liquidity features", gas_end - gas_start),
#         ("Flow imbalance features", flow_end - flow_start),
#         ("Wallet concentration metrics", wallet_conc_end - wallet_conc_start),
#         ("Hour entropy", hour_end - hour_start),
#         ("Price and volatility metrics", price_end - price_start),
#         ("Time-weighted features", tw_end - tw_start),
#         ("Collect lazy results", collect_end - collect_start),
#         ("Join all features", merge_end - merge_start),
#         ("Final processing", final_end - final_start),
#         ("Total processing time", total_end_time - total_start_time)
#     ]
    
#     timing_data.sort(key=lambda x: x[1], reverse=True)
#     for operation, duration in timing_data:
#         print(f"{operation}: {duration:.4f} seconds ({(duration/(total_end_time - total_start_time))*100:.2f}%)")
    
#     return result.to_pandas()

# start_time = datetime.now()
# mint_transactions = fast_polars_process(transactions_df.iloc[:1000000], n_jobs=4)
# end_time = datetime.now()
# elapsed_time = end_time - start_time
# print(f'Total execution time: {elapsed_time}')

In [15]:
# import polars as pl
# import numpy as np
# from datetime import datetime
# import concurrent.futures
# import time

# def fast_polars_process(transactions_df, n_jobs=4):
#     """Fully optimized version using Polars LazyFrames for all operations"""
#     total_start_time = time.time()
#     print(f"Starting Polars transaction processing at {datetime.now()}")
    
#     # Convert to LazyFrame
#     convert_start = time.time()
#     if not isinstance(transactions_df, pl.LazyFrame):
#         if not isinstance(transactions_df, pl.DataFrame):
#             transactions_df = pl.from_pandas(transactions_df)
#         lf = transactions_df.lazy()
#     else:
#         lf = transactions_df
#     convert_end = time.time()
#     print(f"Time to convert to LazyFrame: {convert_end - convert_start:.4f} seconds")
    
#     # Ensure datetime conversion
#     datetime_start = time.time()
#     lf = lf.with_columns(
#         pl.col("block_time").str.to_datetime(strict=False)
#     )
#     datetime_end = time.time()
#     print(f"Time to convert block_time to datetime: {datetime_end - datetime_start:.4f} seconds")
    
#     print("Calculating aggregated features...")
    
#     # Basic counts
#     counts_start = time.time()
#     counts = (
#         lf.group_by("base_coin")
#         .agg(
#             pl.len().alias("total_txs"),
#             pl.n_unique("tx_idx").alias("unique_tx_indices"),
#             pl.n_unique("signing_wallet").alias("unique_wallets")
#         )
#     )
#     counts_end = time.time()
#     print(f"Time to compute basic counts: {counts_end - counts_start:.4f} seconds")
    
#     # Direction-based metrics
#     dir_start = time.time()
#     direction_counts = (
#         lf.group_by("base_coin")
#         .agg(
#             pl.when(pl.col("direction") == "buy")
#               .then(1)
#               .otherwise(0)
#               .sum()
#               .alias("buy_count"),
#             pl.when(pl.col("direction") == "sell")
#               .then(1)
#               .otherwise(0)
#               .sum()
#               .alias("sell_count")
#         )
#     )
#     dir_end = time.time()
#     print(f"Time to compute direction-based metrics: {dir_end - dir_start:.4f} seconds")
    
#     # Wallet counts (lazy)
#     wallet_start = time.time()
#     buy_wallets = (
#         lf.filter(pl.col("direction") == "buy")
#         .group_by("base_coin")
#         .agg(pl.n_unique("signing_wallet").alias("unique_buy_wallets"))
#     )
    
#     sell_wallets = (
#         lf.filter(pl.col("direction") == "sell")
#         .group_by("base_coin")
#         .agg(pl.n_unique("signing_wallet").alias("unique_sell_wallets"))
#     )
#     wallet_end = time.time()
#     print(f"Time to compute wallet counts: {wallet_end - wallet_start:.4f} seconds")
    
#     # Base/quote statistics (lazy)
#     stats_start = time.time()
#     base_stats = (
#         lf.group_by("base_coin")
#         .agg(
#             pl.col("base_coin_amount").sum().alias("base_sum"),
#             pl.col("base_coin_amount").mean().alias("base_mean"),
#             pl.col("base_coin_amount").max().alias("base_max"),
#             pl.col("base_coin_amount").min().alias("base_min"),
#             pl.col("base_coin_amount").std().alias("base_std")
#         )
#     )
    
#     quote_stats = (
#         lf.group_by("base_coin")
#         .agg(
#             pl.col("quote_coin_amount").sum().alias("quote_sum"),
#             pl.col("quote_coin_amount").mean().alias("quote_mean"),
#             pl.col("quote_coin_amount").max().alias("quote_max"),
#             pl.col("quote_coin_amount").min().alias("quote_min"),
#             pl.col("quote_coin_amount").std().alias("quote_std")
#         )
#     )
#     stats_end = time.time()
#     print(f"Time to compute base/quote statistics: {stats_end - stats_start:.4f} seconds")
    
#     # Time-based features (lazy)
#     time_feat_start = time.time()
#     time_stats = (
#         lf.group_by("base_coin")
#         .agg(
#             pl.col("block_time").min().alias("first_time"),
#             pl.col("block_time").max().alias("last_time")
#         )
#         .with_columns(
#             (pl.col("last_time") - pl.col("first_time"))
#             .dt.total_seconds()
#             .alias("activity_duration_sec")
#         )
#         .join(counts.select("base_coin", "total_txs"), on="base_coin")
#         .with_columns(
#             (pl.col("total_txs") / (pl.col("activity_duration_sec") + 1e-6))
#             .alias("tx_per_sec")
#         )
#     )
#     time_feat_end = time.time()
#     print(f"Time to compute time-based features: {time_feat_end - time_feat_start:.4f} seconds")
    
#     # Slot features (lazy)
#     slot_start = time.time()
#     slot_stats = (
#         lf.group_by("base_coin")
#         .agg(
#             pl.col("slot").min().alias("first_slot"),
#             pl.col("slot").max().alias("last_slot")
#         )
#         .with_columns(
#             (pl.col("last_slot") - pl.col("first_slot")).alias("slot_span")
#         )
#     )
    
#     # Early transactions (lazy)
#     early_txs = (
#         lf.group_by("base_coin")
#         .agg(
#             pl.col("tx_idx")
#             .filter(pl.col("slot") <= pl.col("slot").min() + 100)
#             .count()
#             .alias("early_txs")
#         )
#     )
#     slot_end = time.time()
#     print(f"Time to compute slot features: {slot_end - slot_start:.4f} seconds")
    
#     # Balance features (lazy)
#     balance_start = time.time()
#     sorted_lf = lf.sort(["base_coin", "slot"])
#     first_values = (
#         sorted_lf.group_by("base_coin")
#         .agg(
#             pl.col("virtual_token_balance_after").first().alias("first_balance"),
#             pl.col("virtual_sol_balance_after").first().alias("first_sol")
#         )
#     )
#     last_values = (
#         sorted_lf.group_by("base_coin")
#         .agg(
#             pl.col("virtual_token_balance_after").last().alias("last_balance"),
#             pl.col("virtual_sol_balance_after").last().alias("last_sol")
#         )
#     )
#     balance_features = (
#         first_values.join(last_values, on="base_coin")
#         .with_columns(
#             (pl.col("last_sol") - pl.col("first_sol")).alias("sol_change")
#         )
#     )
    
#     # Balance volatility (lazy)
#     balance_vol = (
#         lf.group_by("base_coin")
#         .agg(
#             pl.col("virtual_token_balance_after").std().alias("balance_volatility"),
#             pl.col("virtual_token_balance_after").max().alias("max_balance"),
#             pl.col("virtual_token_balance_after").min().alias("min_balance")
#         )
#     )
#     balance_end = time.time()
#     print(f"Time to compute balance features: {balance_end - balance_start:.4f} seconds")
    
#     # Gas features (lazy)
#     gas_start = time.time()
#     gas_stats = (
#         lf.group_by("base_coin")
#         .agg(
#             pl.col("provided_gas_fee").sum().alias("total_gas_fees"),
#             pl.col("provided_gas_fee").mean().alias("avg_gas_fee"),
#             pl.col("provided_gas_fee").std().alias("std_gas_fee"),
#             pl.col("provided_gas_fee").max().alias("max_gas_fee"),
#             pl.col("provided_gas_fee").min().alias("min_gas_fee"),
#             pl.col("provided_gas_limit").mean().alias("avg_gas_limit"),
#             pl.col("consumed_gas").mean().alias("avg_consumed_gas"),
#             pl.col("consumed_gas").sum().alias("total_consumed_gas"),
#             pl.col("fee").sum().alias("total_fee"),
#             (pl.col("provided_gas_limit") - pl.col("consumed_gas")).mean().alias("limit_utilization")
#         )
#     )
    
#     # Liquidity stats (lazy)
#     liq_stats = (
#         lf.group_by("base_coin")
#         .agg(
#             pl.col("virtual_sol_balance_after").mean().alias("liq_virtual_sol_balance_after_mean"),
#             pl.col("quote_coin_amount").sum().alias("liq_quote_coin_amount")
#         )
#     )
    
#     # Gas metrics (lazy)
#     gas_metrics = (
#         gas_stats.join(quote_stats.select(["base_coin", "quote_sum"]), on="base_coin")
#         .with_columns([
#             (pl.col("total_fee") / (pl.col("quote_sum") + 1e-6)).alias("fee_per_sol"),
#             pl.col("avg_consumed_gas").alias("gas_per_tx"),
#             (pl.col("quote_sum") / (pl.col("total_consumed_gas") + 1e-6)).alias("gas_efficiency")
#         ])
#     )
#     gas_end = time.time()
#     print(f"Time to compute gas and liquidity features: {gas_end - gas_start:.4f} seconds")
    
#     # Flow imbalance (lazy)
#     flow_start = time.time()
#     flow_imbalance = (
#         lf.group_by("base_coin")
#         .agg(
#             pl.col("quote_coin_amount").filter(pl.col("direction") == "buy").sum().alias("buy_quote_amount"),
#             pl.col("quote_coin_amount").filter(pl.col("direction") == "sell").sum().alias("sell_quote_amount")
#         )
#         .with_columns(
#             ((pl.col("buy_quote_amount") - pl.col("sell_quote_amount")) / 
#              (pl.col("buy_quote_amount") + pl.col("sell_quote_amount") + 1e-6)).alias("flow_imbalance")
#         )
#         .select(["base_coin", "flow_imbalance"])
#     )
#     flow_end = time.time()
#     print(f"Time to compute flow imbalance features: {flow_end - flow_start:.4f} seconds")
    
#     # Collect necessary data for custom functions
#     print("Collecting data for custom functions...")
#     collect_start = time.time()
#     unique_mints = counts.collect()["base_coin"].to_list()
#     collect_end = time.time()
#     print(f"Time to collect unique mints: {collect_end - collect_start:.4f} seconds")
    
#     # Wallet concentration metrics - optimized version
#     wallet_conc_start = time.time()
#     print("Calculating wallet concentration metrics...")

#     # Pre-calculate wallet volumes for all mints at once
#     wallet_volumes = (
#         lf.group_by(["base_coin", "signing_wallet"])
#         .agg(pl.col("base_coin_amount").sum().alias("volume"))
#         .collect()
#     )

#     # Function to calculate Gini and top5 share for a single mint using pre-calculated data
#     def calculate_concentration_metrics_optimized(mint):
#         mint_volumes = wallet_volumes.filter(pl.col("base_coin") == mint)
        
#         if mint_volumes.height <= 1:
#             return {"base_coin": mint, "gini_coeff": 0.0, "top5_wallet_share": 0.0}
        
#         sorted_values = mint_volumes.sort("volume")["volume"].to_numpy()
#         n = len(sorted_values)
        
#         # Gini coefficient calculation
#         cumulative_values = np.cumsum(sorted_values)
#         total = cumulative_values[-1]
#         if total <= 0:
#             return {"base_coin": mint, "gini_coeff": 0.0, "top5_wallet_share": 0.0}
        
#         # Faster Gini calculation using cumulative sum
#         gini_coeff = (n + 1 - 2 * np.sum(cumulative_values) / total) / n
        
#         # Top wallet share
#         top_k = min(5, n)
#         top5_wallet_share = np.sum(sorted_values[-top_k:]) / total
        
#         return {"base_coin": mint, "gini_coeff": float(gini_coeff), "top5_wallet_share": float(top5_wallet_share)}

#     # Process in parallel using the pre-calculated data
#     with concurrent.futures.ThreadPoolExecutor(max_workers=n_jobs) as executor:
#         concentration_results = list(executor.map(calculate_concentration_metrics_optimized, unique_mints))
#     concentration_df = pl.DataFrame(concentration_results)

#     wallet_conc_end = time.time()
#     print(f"Time to compute wallet concentration metrics: {wallet_conc_end - wallet_conc_start:.4f} seconds")

#     # Hour entropy - optimized version
#     hour_start = time.time()
#     print("Calculating hour entropy...")

#     # Calculate hours for all records at once
#     hours_df = (
#         lf.select([
#             pl.col("base_coin"),
#             pl.col("block_time").dt.hour().alias("hour")
#         ])
#         .collect()
#     )

#     # Calculate hour counts for all mints at once
#     hour_counts = (
#         hours_df.group_by(["base_coin", "hour"])
#         .agg(pl.len().alias("count"))
#     )

#     # Calculate entropy for each mint
#     mint_entropy = []
#     for mint in unique_mints:
#         mint_hours = hour_counts.filter(pl.col("base_coin") == mint)
#         if mint_hours.height == 0:
#             mint_entropy.append({"base_coin": mint, "hour_entropy": 0.0})
#             continue
        
#         total = mint_hours["count"].sum()
#         if total == 0:
#             mint_entropy.append({"base_coin": mint, "hour_entropy": 0.0})
#             continue
        
#         probs = mint_hours["count"] / total
#         non_zero_probs = probs.filter(probs > 0).to_numpy()
        
#         if len(non_zero_probs) == 0:
#             mint_entropy.append({"base_coin": mint, "hour_entropy": 0.0})
#             continue
        
#         entropy = -np.sum(non_zero_probs * np.log2(non_zero_probs))
#         mint_entropy.append({"base_coin": mint, "hour_entropy": float(entropy)})
        
#     hour_entropy_df = pl.DataFrame(mint_entropy)
#     hour_end = time.time()
#     print(f"Time to compute hour entropy: {hour_end - hour_start:.4f} seconds")

#     # Price and volatility - optimized version
#     price_start = time.time()
#     print("Calculating price and volatility metrics...")

#     # Calculate prices for all records at once
#     price_df_all = (
#         lf.with_columns(
#             (pl.col("quote_coin_amount") / pl.col("base_coin_amount"))
#             .replace(0, None)
#             .alias("price")
#         )
#         .select(["base_coin", "slot", "price"])
#         .collect()
#         .filter(
#             (pl.col("price").is_not_nan()) & 
#             (pl.col("price").is_not_null()) & 
#             (pl.col("price").is_finite())
#         )
#         .sort(["base_coin", "slot"])
#     )

#     # Calculate price metrics for each mint
#     price_metrics = []
#     for mint in unique_mints:
#         mint_prices = price_df_all.filter(pl.col("base_coin") == mint)
        
#         if mint_prices.height == 0:
#             price_metrics.append({
#                 "base_coin": mint,
#                 "avg_price": 0.0,
#                 "price_vol_mean": 0.0,
#                 "price_vol_max": 0.0,
#                 "price_slippage": 0.0
#             })
#             continue
        
#         prices = mint_prices["price"].to_numpy()
#         avg_price = np.mean(prices)
#         price_min = np.min(prices)
#         price_max = np.max(prices)
        
#         # Calculate rolling standard deviation without pandas
#         if len(prices) >= 5:
#             # Calculate rolling window standard deviations with numpy
#             window_size = 5
#             rolling_stds = []
#             for i in range(len(prices) - window_size + 1):
#                 window = prices[i:i+window_size]
#                 rolling_stds.append(np.std(window))
#             if rolling_stds:
#                 price_vol_mean = np.mean(rolling_stds)
#                 price_vol_max = np.max(rolling_stds)
#             else:
#                 price_vol_mean = np.std(prices)
#                 price_vol_max = price_vol_mean
#         else:
#             price_vol_mean = np.std(prices)
#             price_vol_max = price_vol_mean
        
#         price_slippage = (price_max - price_min) / (avg_price + 1e-6) if avg_price != 0 else 0.0
        
#         price_metrics.append({
#             "base_coin": mint,
#             "avg_price": float(avg_price),
#             "price_vol_mean": float(price_vol_mean),
#             "price_vol_max": float(price_vol_max),
#             "price_slippage": float(price_slippage)
#         })

#     price_df = pl.DataFrame(price_metrics)
#     price_end = time.time()
#     print(f"Time to compute price and volatility metrics: {price_end - price_start:.4f} seconds")

#     # Time-weighted features - optimized version
#     tw_start = time.time()
#     print("Calculating time-weighted features...")

#     # Get max slot for each base_coin in one operation
#     max_slots = (
#         lf.group_by("base_coin")
#         .agg(pl.col("slot").max().alias("max_slot"))
#         .collect()
#     )

#     # Create expanded dataframe with time weights
#     tw_expanded = (
#         lf.select([
#             "base_coin", "slot", "base_coin_amount", "quote_coin_amount"
#         ])
#         .collect()
#         .join(max_slots, on="base_coin", how="left")
#         .with_columns(
#             (0.1 * (pl.col("slot") - pl.col("max_slot")))
#             .exp()
#             .alias("time_weight")
#         )
#     )

#     # Calculate time-weighted sums for all mints at once
#     tw_results = (
#         tw_expanded.group_by("base_coin")
#         .agg([
#             (pl.col("base_coin_amount") * pl.col("time_weight")).sum().alias("tw_base_coin_amount"),
#             (pl.col("quote_coin_amount") * pl.col("time_weight")).sum().alias("tw_quote_coin_amount")
#         ])
#         .with_columns([
#             pl.col("tw_base_coin_amount").fill_null(0.0).cast(pl.Float64),
#             pl.col("tw_quote_coin_amount").fill_null(0.0).cast(pl.Float64)
#         ])
#     )

#     tw_df = tw_results
#     tw_end = time.time()
#     print(f"Time to compute time-weighted features: {tw_end - tw_start:.4f} seconds")
    
#     # Collect all lazy results
#     print("Collecting lazy results...")
#     collect_start = time.time()
#     collected = {
#         "counts": counts.collect(),
#         "direction_counts": direction_counts.collect(),
#         "buy_wallets": buy_wallets.collect(),
#         "sell_wallets": sell_wallets.collect(),
#         "base_stats": base_stats.collect(),
#         "quote_stats": quote_stats.collect(),
#         "time_stats": time_stats.collect(),
#         "slot_stats": slot_stats.collect(),
#         "early_txs": early_txs.collect(),
#         "balance_features": balance_features.collect(),
#         "balance_vol": balance_vol.collect(),
#         "gas_metrics": gas_metrics.collect(),
#         "liq_stats": liq_stats.collect(),
#         "flow_imbalance": flow_imbalance.collect()
#     }
#     collect_end = time.time()
#     print(f"Time to collect all lazy results: {collect_end - collect_start:.4f} seconds")
    
#     # Merge all features
#     print("Merging all features...")
#     merge_start = time.time()
#     result = collected["counts"].join(collected["direction_counts"], on="base_coin", how="left")
#     result = result.with_columns(
#         (pl.col("buy_count") / pl.col("total_txs")).alias("buy_ratio")
#     )
    
#     dfs_to_join = [
#         collected["buy_wallets"],
#         collected["sell_wallets"],
#         collected["base_stats"],
#         collected["quote_stats"],
#         collected["time_stats"].select(["base_coin", "activity_duration_sec", "first_time", "last_time", "tx_per_sec"]),
#         collected["slot_stats"],
#         collected["early_txs"],
#         collected["balance_features"],
#         collected["balance_vol"],
#         collected["gas_metrics"],
#         concentration_df,
#         hour_entropy_df,
#         collected["flow_imbalance"],
#         price_df,
#         tw_df,
#         collected["liq_stats"]
#     ]
    
#     for df in dfs_to_join:
#         result = result.join(df, on="base_coin", how="left")
#     merge_end = time.time()
#     print(f"Time to join all features: {merge_end - merge_start:.4f} seconds")
    
#     # Final processing
#     final_start = time.time()
#     result = result.rename({"base_coin": "mint"}).fill_null(0)
#     final_end = time.time()
#     print(f"Time for final processing: {final_end - final_start:.4f} seconds")
    
#     total_end_time = time.time()
#     print(f"Total processing time: {total_end_time - total_start_time:.4f} seconds")
#     print(f"Polars processing complete with {result.height} rows and {len(result.columns)} columns")
    
#     # Summary of time profiling
#     print("\n=== TIMING SUMMARY ===")
#     timing_data = [
#         ("Convert to LazyFrame", convert_end - convert_start),
#         ("Datetime conversion", datetime_end - datetime_start),
#         ("Basic counts", counts_end - counts_start),
#         ("Direction-based metrics", dir_end - dir_start),
#         ("Wallet counts", wallet_end - wallet_start),
#         ("Base/quote statistics", stats_end - stats_start),
#         ("Time-based features", time_feat_end - time_feat_start),
#         ("Slot features", slot_end - slot_start),
#         ("Balance features", balance_end - balance_start),
#         ("Gas and liquidity features", gas_end - gas_start),
#         ("Flow imbalance features", flow_end - flow_start),
#         ("Wallet concentration metrics", wallet_conc_end - wallet_conc_start),
#         ("Hour entropy", hour_end - hour_start),
#         ("Price and volatility metrics", price_end - price_start),
#         ("Time-weighted features", tw_end - tw_start),
#         ("Collect lazy results", collect_end - collect_start),
#         ("Join all features", merge_end - merge_start),
#         ("Final processing", final_end - final_start),
#         ("Total processing time", total_end_time - total_start_time)
#     ]
    
#     timing_data.sort(key=lambda x: x[1], reverse=True)
#     for operation, duration in timing_data:
#         print(f"{operation}: {duration:.4f} seconds ({(duration/(total_end_time - total_start_time))*100:.2f}%)")
    
#     return result.to_pandas()

# start_time = datetime.now()
# mint_transactions = fast_polars_process(transactions_df.iloc[:100000], n_jobs=4)
# end_time = datetime.now()
# elapsed_time = end_time - start_time
# print(f'Total execution time: {elapsed_time}')

In [16]:
# import polars as pl
# import numpy as np
# from datetime import datetime

# def fast_polars_process(transactions_df, n_jobs=4):
#     total_start_time = time.time()
#     print(f"Starting Polars transaction processing at {datetime.now()}")

#     # Convert to LazyFrame
#     if not isinstance(transactions_df, pl.LazyFrame):
#         if not isinstance(transactions_df, pl.DataFrame):
#             transactions_df = pl.from_pandas(transactions_df)
#         lf = transactions_df.lazy()
#     else:
#         lf = transactions_df

#     # Ensure datetime conversion
#     lf = lf.with_columns(
#         pl.col("block_time").str.to_datetime(strict=False)
#     )

#     # Precompute common group_by operations
#     base_group = lf.group_by("base_coin")

#     # Vectorized wallet concentration metrics
# # Vectorized wallet concentration metrics (FIXED HERE)
#     wallet_concentration = (
#         lf.group_by(["base_coin", "signing_wallet"])
#         .agg(pl.col("base_coin_amount").sum().alias("volume"))
#         .sort(["base_coin", "volume"])  # Sort within each base_coin group
#         .with_columns([
#             pl.col("volume").rank("ordinal", descending=False).over("base_coin").alias("rank"),
#             pl.col("volume").sum().over("base_coin").alias("total_volume")
#         ])
#         .filter(pl.col("total_volume") > 0)
#         .with_columns([
#         # FIXED: Use cumsum() instead of cumulative_sum()
#             (pl.col("volume").cum_sum().over("base_coin").last() / pl.col("total_volume")).alias("cumsum_ratio"),
#             (pl.col("volume") * pl.col("rank")).sum().over("base_coin").alias("rank_volume_sum")
#         ])
#         .group_by("base_coin")
#         .agg([
#             ((pl.count() + 1 - 2 * pl.col("rank_volume_sum") / pl.col("total_volume")) 
#              / pl.count()).alias("gini_coeff"),
#             (pl.col("volume").top_k(5).sum() / pl.col("total_volume")).alias("top5_wallet_share")
#         ])
#         .fill_null(0)
#     )
#     # Vectorized hour entropy
#     hour_entropy = (
#         lf
#         .select([
#             "base_coin",
#             pl.col("block_time").dt.hour().alias("hour")
#         ])
#         .group_by(["base_coin", "hour"])
#         .agg(
#             pl.len().alias("count")                           # use pl.len() not pl.count()
#         )
#         .with_columns(
#             (pl.col("count") / pl.sum("count").over("base_coin"))
#               .alias("prob")
#         )
#         .filter(pl.col("prob") > 0)
#         .group_by("base_coin")
#         .agg(
#         # compute -∑ p·log2(p) via .log(2) + .sum()
#             (- (pl.col("prob") * pl.col("prob").log(2))).sum()
#              .alias("hour_entropy")
#         )
#         .fill_null(0)
#     )


#     # Vectorized price and volatility
#     price_volatility = (
#         lf.with_columns(
#             (pl.col("quote_coin_amount") / pl.col("base_coin_amount"))
#             .replace(0, None)
#             .alias("price")
#         )
#         .select(["base_coin", "slot", "price"])
#         .filter(pl.col("price").is_finite())
#         .sort(["base_coin", "slot"])
#         .group_by("base_coin")
#         .agg([
#             pl.col("price").mean().alias("avg_price"),
#             pl.col("price").rolling_std(window_size=5, min_periods=1).mean().alias("price_vol_mean"),
#             pl.col("price").rolling_std(window_size=5, min_periods=1).max().alias("price_vol_max"),
#             ((pl.col("price").max() - pl.col("price").min()) 
#              / (pl.col("price").mean() + 1e-6)).alias("price_slippage")
#         ])
#         .fill_null(0)
#     )

#     # Time-weighted features with JIT
#     time_weighted = (
#         lf.group_by("base_coin")
#         .agg(pl.col("slot").max().alias("max_slot"))
#         .join(lf.select(["base_coin", "slot", "base_coin_amount", "quote_coin_amount"]), 
#               on="base_coin", how="left")
#         .with_columns(
#             (0.1 * (pl.col("slot") - pl.col("max_slot")))
#             .exp()
#             .alias("time_weight")
#         )
#         .group_by("base_coin")
#         .agg([
#             (pl.col("base_coin_amount") * pl.col("time_weight")).sum().alias("tw_base_coin_amount"),
#             (pl.col("quote_coin_amount") * pl.col("time_weight")).sum().alias("tw_quote_coin_amount")
#         ])
#         .collect()  # Use JIT here for large aggregations
#         .fill_null(0)
#     )

#     # Combine all features
#     all_features = [
#         wallet_concentration,
#         hour_entropy,
#         price_volatility,
#         time_weighted.lazy()
#     ]

#     # Final join (simplified for brevity)
#     result = (
#         base_group.agg(
#             pl.len().alias("total_txs"),
#             pl.n_unique("tx_idx").alias("unique_tx_indices"),
#             pl.n_unique("signing_wallet").alias("unique_wallets")
#         )
#         .join(wallet_concentration, on="base_coin", how="left")
#         .join(hour_entropy, on="base_coin", how="left")
#         .join(price_volatility, on="base_coin", how="left")
#         .join(time_weighted.lazy(), on="base_coin", how="left")
#     )

#     # Final collection with JIT where beneficial
#     final_df = result.collect()
#     return final_df.to_pandas()

# # Ensure Polars is updated to >=0.18.0
# mint_transactions_2 = fast_polars_process(transactions_df.iloc[:100000], n_jobs=4)

In [17]:
import polars as pl
import time
from datetime import datetime

def fast_polars_process(transactions_df, n_jobs=4):
    """Fully vectorized Polars pipeline computing ~61 mint-level features with detailed timing."""
    t0 = time.time()
    print(f"[Overall Start] {datetime.now()}")
    
    # Prepare LazyFrame
    t1 = time.time()
    if isinstance(transactions_df, pl.DataFrame):
        lf = transactions_df.lazy()
    else:
        lf = pl.from_pandas(transactions_df).lazy()
    print(f"[Prepare LazyFrame] {time.time() - t1:.3f}s")
    
    # Ensure datetime conversion
    t2 = time.time()
    lf = lf.with_columns(
        pl.col("block_time").str.to_datetime(strict=False)
    )
    print(f"[Datetime Conversion] {time.time() - t2:.3f}s")
    
    # Base grouping
    t3 = time.time()
    base = lf.group_by("base_coin")
    print(f"[Group By base_coin] {time.time() - t3:.3f}s")
    
    # 1. Basic counts & direction
    t4 = time.time()
    counts = base.agg(
        pl.len().alias("total_txs"),
        pl.n_unique("tx_idx").alias("unique_tx_indices"),
        pl.n_unique("signing_wallet").alias("unique_wallets")
    )
    
    raw_dir = base.agg(
        pl.when(pl.col("direction") == "buy").then(1).otherwise(0).sum().alias("buy_count"),
        pl.when(pl.col("direction") == "sell").then(1).otherwise(0).sum().alias("sell_count")
    )
    
    dir_stats = (
        raw_dir
        .join(counts.select(["base_coin", "total_txs"]), on="base_coin")
        .with_columns(
            (pl.col("buy_count") / pl.col("total_txs")).alias("buy_ratio")
        )
        .rename({"total_txs": "total_txs_dir"})  # Rename to avoid conflict
    )
    print(f"[Counts & Direction] {time.time() - t4:.3f}s")
    
    # 2. Wallet counts
    t5 = time.time()
    buy_wallets = (
        lf.filter(pl.col("direction") == "buy")
          .group_by("base_coin").agg(pl.n_unique("signing_wallet").alias("unique_buy_wallets"))
    )
    sell_wallets = (
        lf.filter(pl.col("direction") == "sell")
          .group_by("base_coin").agg(pl.n_unique("signing_wallet").alias("unique_sell_wallets"))
    )
    print(f"[Wallet Counts] {time.time() - t5:.3f}s")
    
    # 3. Base/Quote stats
    t6 = time.time()
    base_stats = base.agg(
        pl.col("base_coin_amount").sum().alias("base_sum"),
        pl.col("base_coin_amount").mean().alias("base_mean"),
        pl.col("base_coin_amount").max().alias("base_max"),
        pl.col("base_coin_amount").min().alias("base_min"),
        pl.col("base_coin_amount").std().alias("base_std")
    )
    quote_stats = base.agg(
        pl.col("quote_coin_amount").sum().alias("quote_sum"),
        pl.col("quote_coin_amount").mean().alias("quote_mean"),
        pl.col("quote_coin_amount").max().alias("quote_max"),
        pl.col("quote_coin_amount").min().alias("quote_min"),
        pl.col("quote_coin_amount").std().alias("quote_std")
    )
    print(f"[Base/Quote Stats] {time.time() - t6:.3f}s")
    
    # 4. Time-based features
    t7 = time.time()
    time_stats = (
        base.agg(
            pl.col("block_time").min().alias("first_time"),
            pl.col("block_time").max().alias("last_time")
        )
        .with_columns(
            (pl.col("last_time") - pl.col("first_time")).dt.total_seconds().alias("activity_duration_sec")
        )
        .join(counts, on="base_coin")
        .with_columns(
            (pl.col("total_txs") / (pl.col("activity_duration_sec") + 1e-6)).alias("tx_per_sec")
        )
        .rename({"total_txs": "total_txs_time"})  # Rename to avoid conflict
    )
    print(f"[Time Features] {time.time() - t7:.3f}s")
    
    # 5. Slot features
    t8 = time.time()
    slot_stats = (
        base.agg(
            pl.col("slot").min().alias("first_slot"),
            pl.col("slot").max().alias("last_slot")
        )
        .with_columns(
            (pl.col("last_slot") - pl.col("first_slot")).alias("slot_span")
        )
    )
    early_txs = lf.with_columns(
        pl.when(pl.col("slot") <= pl.col("slot").min() + 100)
          .then(1).otherwise(0).alias("is_early")
    ).group_by("base_coin").agg(pl.col("is_early").sum().alias("early_txs"))
    print(f"[Slot Features] {time.time() - t8:.3f}s")
    
    # 6. Balance features
    t9 = time.time()
    sorted_lf = lf.sort(["base_coin", "slot"])
    balance_feats = (
        sorted_lf.group_by("base_coin").agg(
            pl.col("virtual_token_balance_after").first().alias("first_balance"),
            pl.col("virtual_token_balance_after").last().alias("last_balance"),
            pl.col("virtual_sol_balance_after").first().alias("first_sol"),
            pl.col("virtual_sol_balance_after").last().alias("last_sol")
        )
        .with_columns((pl.col("last_sol") - pl.col("first_sol")).alias("sol_change"))
    )
    balance_vol = base.agg(
        pl.col("virtual_token_balance_after").std().alias("balance_volatility"),
        pl.col("virtual_token_balance_after").max().alias("max_balance"),
        pl.col("virtual_token_balance_after").min().alias("min_balance")
    )
    print(f"[Balance Features] {time.time() - t9:.3f}s")
    
    # 7. Gas & Liquidity
    t10 = time.time()
    gas = base.agg(
        pl.col("provided_gas_fee").sum().alias("total_gas_fees"),
        pl.col("provided_gas_fee").mean().alias("avg_gas_fee"),
        pl.col("provided_gas_fee").std().alias("std_gas_fee"),
        pl.col("provided_gas_fee").max().alias("max_gas_fee"),
        pl.col("provided_gas_fee").min().alias("min_gas_fee"),
        pl.col("provided_gas_limit").mean().alias("avg_gas_limit"),
        pl.col("consumed_gas").mean().alias("avg_consumed_gas"),
        pl.col("consumed_gas").sum().alias("total_consumed_gas"),
        pl.col("fee").sum().alias("total_fee"),
        ((pl.col("provided_gas_limit") - pl.col("consumed_gas")).mean()).alias("limit_utilization")
    )
    liq = base.agg(
        pl.col("virtual_sol_balance_after").mean().alias("liq_virtual_sol_balance_after_mean"),
        pl.col("quote_coin_amount").sum().alias("liq_quote_coin_amount")
    )
    gas_metrics = gas.join(quote_stats, on="base_coin").with_columns([
        (pl.col("total_fee") / (pl.col("quote_sum") + 1e-6)).alias("fee_per_sol"),
        pl.col("avg_consumed_gas").alias("gas_per_tx"),
        (pl.col("quote_sum") / (pl.col("total_consumed_gas") + 1e-6)).alias("gas_efficiency")
    ])
    print(f"[Gas & Liquidity] {time.time() - t10:.3f}s")
    
    # 8. Flow Imbalance
    t11 = time.time()
    flow = base.agg(
        pl.col("quote_coin_amount").filter(pl.col("direction") == "buy").sum().alias("buy_quote_amount"),
        pl.col("quote_coin_amount").filter(pl.col("direction") == "sell").sum().alias("sell_quote_amount")
    ).with_columns(
        ((pl.col("buy_quote_amount") - pl.col("sell_quote_amount")) /
         (pl.col("buy_quote_amount") + pl.col("sell_quote_amount") + 1e-6)).alias("flow_imbalance")
    )
    print(f"[Flow Imbalance] {time.time() - t11:.3f}s")
    
    # 9. Wallet Concentration
    t12 = time.time()
    # wallet_conc = (
    #     lf.group_by(["base_coin", "signing_wallet"])
    #       .agg(pl.col("base_coin_amount").sum().alias("volume"))
    #       .sort(["base_coin", "volume"])
    #       .with_columns([
    #           pl.col("volume").cum_sum().over("base_coin").alias("cum_vol"),
    #           pl.col("volume").sum().over("base_coin").alias("total_vol"),
    #           pl.col("volume").rank("ordinal").over("base_coin").alias("rank")
    #       ])
    #       .group_by("base_coin").agg([
    #           ((pl.len() + 1 - 2 * (pl.col("cum_vol") / pl.col("total_vol")).sum()) / pl.len()).alias("gini_coeff"),
    #           (pl.col("volume").top_k(5).sum() / pl.col("total_vol")).alias("top5_wallet_share")
    #       ])
    # )
    wallet_conc = (
        lf.group_by(["base_coin", "signing_wallet"])
        .agg(pl.sum("base_coin_amount").alias("wallet_volume"))
        .sort(["base_coin", "wallet_volume"])
        .group_by("base_coin")
        .agg(
            (1 - 2 * (pl.cum_sum("wallet_volume") / pl.sum("wallet_volume")).sum() / pl.len() + 1/pl.len()).alias("gini_coeff"),
            (pl.col("wallet_volume").top_k(5).sum() / pl.sum("wallet_volume")).alias("top5_wallet_share")
        )
    )

    # print(wallet_conc)
    print(f"[Wallet Concentration] {time.time() - t12:.3f}s")
    
    # 10. Hour Entropy
    t13 = time.time()
    hour_ent = (
        lf.select([pl.col("base_coin"), pl.col("block_time").dt.hour().alias("hour")])
          .group_by(["base_coin", "hour"]).agg(pl.len().alias("cnt"))
          .with_columns((pl.col("cnt") / pl.sum("cnt").over("base_coin")).alias("p"))
          .filter(pl.col("p") > 0)
          .group_by("base_coin").agg((-(pl.col("p") * pl.col("p").log(2))).sum().alias("hour_entropy"))
    )
    print(f"[Hour Entropy] {time.time() - t13:.3f}s")
    
    # 11. Price Volatility
    t14 = time.time()
    price_vol = (
        lf.with_columns((pl.col("quote_coin_amount") / pl.col("base_coin_amount")).alias("price"))
          .filter(pl.col("price").is_finite())
          .group_by("base_coin").agg([
              pl.col("price").mean().alias("avg_price"),
              pl.col("price").rolling_std(window_size=5, min_periods=1).mean().alias("price_vol_mean"),
              pl.col("price").rolling_std(window_size=5, min_periods=1).max().alias("price_vol_max"),
              ((pl.col("price").max() - pl.col("price").min()) / (pl.col("price").mean() + 1e-6)).alias("price_slippage")
          ])
    )
    print(f"[Price Volatility] {time.time() - t14:.3f}s")
    
    # 12. Time-Weighted Features
    t15 = time.time()
    max_slots = lf.group_by("base_coin").agg(pl.col("slot").max().alias("max_slot"))
    tw = (
        lf.join(max_slots, on="base_coin")
          .with_columns((0.1 * (pl.col("slot") - pl.col("max_slot"))).exp().alias("w"))
          .group_by("base_coin").agg([
              (pl.col("base_coin_amount") * pl.col("w")).sum().alias("tw_base_coin_amount"),
              (pl.col("quote_coin_amount") * pl.col("w")).sum().alias("tw_quote_coin_amount")
          ])
    )
    print(f"[Time-Weighted] {time.time() - t15:.3f}s")
    
    # Final join & collect
    t16 = time.time()
    df = (
        counts
        .join(dir_stats, on="base_coin", how="left")
        .join(buy_wallets, on="base_coin", how="left")
        .join(sell_wallets, on="base_coin", how="left")
        .join(base_stats, on="base_coin", how="left")
        .join(quote_stats, on="base_coin", how="left")
        .join(time_stats, on="base_coin", how="left")
        .join(slot_stats, on="base_coin", how="left")
        .join(early_txs, on="base_coin", how="left")
        .join(balance_feats, on="base_coin", how="left")
        .join(balance_vol, on="base_coin", how="left")
        .join(gas_metrics, on="base_coin", how="left")
        .join(liq, on="base_coin", how="left")
        .join(flow, on="base_coin", how="left")
        .join(wallet_conc, on="base_coin", how="left")
        .join(hour_ent, on="base_coin", how="left")
        .join(price_vol, on="base_coin", how="left")
        .join(tw, on="base_coin", how="left")
        .collect()
    )
    pdf = df.rename({"base_coin": "mint"}).fill_null(0).to_pandas()
    print(f"[Final Join & Collect] {time.time() - t16:.3f}s")
    print(f"[Overall Complete] {time.time() - t0:.3f}s, rows={pdf.shape[0]}, cols={pdf.shape[1]}")
    return pdf

mint_transactions = fast_polars_process(transactions_df, n_jobs=4)

[Overall Start] 2025-07-23 12:03:20.194857
[Prepare LazyFrame] 21.900s
[Datetime Conversion] 0.006s
[Group By base_coin] 0.001s
[Counts & Direction] 0.001s
[Wallet Counts] 0.000s
[Base/Quote Stats] 0.000s
[Time Features] 0.001s
[Slot Features] 0.000s
[Balance Features] 0.000s
[Gas & Liquidity] 0.000s
[Flow Imbalance] 0.000s
[Wallet Concentration] 0.000s
[Hour Entropy] 0.000s
[Price Volatility] 0.001s
[Time-Weighted] 0.000s


<ipython-input-17-4a8fe06f882b>:221: DeprecationWarning: the argument `min_periods` for `Expr.rolling_std` is deprecated. It was renamed to `min_samples` in version 1.21.0.
  pl.col("price").rolling_std(window_size=5, min_periods=1).mean().alias("price_vol_mean"),
<ipython-input-17-4a8fe06f882b>:222: DeprecationWarning: the argument `min_periods` for `Expr.rolling_std` is deprecated. It was renamed to `min_samples` in version 1.21.0.
  pl.col("price").rolling_std(window_size=5, min_periods=1).max().alias("price_vol_max"),


[Final Join & Collect] 60.903s
[Overall Complete] 82.819s, rows=1118389, cols=71


In [18]:
pd.set_option('display.max_rows', None)
print(mint_transactions.dtypes)
pd.reset_option('display.max_rows')

mint                                          object
total_txs                                     uint32
unique_tx_indices                             uint32
unique_wallets                                uint32
buy_count                                      int32
sell_count                                     int32
total_txs_dir                                 uint32
buy_ratio                                    float64
unique_buy_wallets                            uint32
unique_sell_wallets                           uint32
base_sum                                       int64
base_mean                                    float64
base_max                                       int64
base_min                                       int64
base_std                                     float64
quote_sum                                      int64
quote_mean                                   float64
quote_max                                      int64
quote_min                                     

In [19]:
# mint_transactions_3[mint_transactions_3['mint'] == 'DdMHaNkPHYAYM7MxAnLtF7nCdnxz3XdCCkqSjiJSpump']

In [20]:
# mint_transactions[mint_transactions['mint'] == 'DdMHaNkPHYAYM7MxAnLtF7nCdnxz3XdCCkqSjiJSpump']

In [21]:
# pd.set_option('display.max_rows', None)
# mint_transactions.nunique()

In [22]:
# pd.set_option('display.max_columns', None)
# mint_transactions.head(3)

In [23]:
# import polars as pl
# import numpy as np
# from datetime import datetime
# import concurrent.futures

# def fast_polars_process(transactions_df, n_jobs=4):
#     """Optimized version of mint processing using Polars operations"""
#     print(f"Starting Polars transaction processing at {datetime.now()}")
    
#     # Convert pandas dataframe to polars if needed
#     if not isinstance(transactions_df, pl.DataFrame):
#         transactions_df = pl.from_pandas(transactions_df)
    
#     # Ensure datetime conversion
#     if transactions_df["block_time"].dtype != pl.Datetime:
#         transactions_df = transactions_df.with_columns(
#             pl.col("block_time").str.to_datetime()
#         )
    
#     print("Calculating aggregated features...")
    
#     # Basic counts (vectorized)
#     counts = (
#         transactions_df
#         .group_by("base_coin")
#         .agg(
#             pl.count().alias("total_txs"),
#             pl.n_unique("tx_idx").alias("unique_tx_indices"),
#             pl.n_unique("signing_wallet").alias("unique_wallets")
#         )
#     )
    
#     # Direction-based metrics
#     direction_counts = (
#         transactions_df
#         .group_by(["base_coin", "direction"])
#         .agg(pl.count().alias("count"))
#         .pivot(index="base_coin", columns="direction", values="count")
#         .fill_null(0)
#     )
    
#     # Ensure buy and sell columns exist
#     if "buy" not in direction_counts.columns:
#         direction_counts = direction_counts.with_columns(pl.lit(0).alias("buy"))
#     if "sell" not in direction_counts.columns:
#         direction_counts = direction_counts.with_columns(pl.lit(0).alias("sell"))
    
#     # Rename columns
#     direction_counts = direction_counts.rename({"buy": "buy_count", "sell": "sell_count"})
    
#     # Wallet counts by direction
#     buy_wallets = (
#         transactions_df
#         .filter(pl.col("direction") == "buy")
#         .group_by("base_coin")
#         .agg(pl.n_unique("signing_wallet").alias("unique_buy_wallets"))
#     )
    
#     sell_wallets = (
#         transactions_df
#         .filter(pl.col("direction") == "sell")
#         .group_by("base_coin")
#         .agg(pl.n_unique("signing_wallet").alias("unique_sell_wallets"))
#     )
    
#     # Calculate base and quote statistics
#     base_stats = (
#         transactions_df
#         .group_by("base_coin")
#         .agg(
#             pl.col("base_coin_amount").sum().alias("base_sum"),
#             pl.col("base_coin_amount").mean().alias("base_mean"),
#             pl.col("base_coin_amount").max().alias("base_max"),
#             pl.col("base_coin_amount").min().alias("base_min"),
#             pl.col("base_coin_amount").std().alias("base_std")
#         )
#     )
    
#     quote_stats = (
#         transactions_df
#         .group_by("base_coin")
#         .agg(
#             pl.col("quote_coin_amount").sum().alias("quote_sum"),
#             pl.col("quote_coin_amount").mean().alias("quote_mean"),
#             pl.col("quote_coin_amount").max().alias("quote_max"),
#             pl.col("quote_coin_amount").min().alias("quote_min"),
#             pl.col("quote_coin_amount").std().alias("quote_std")
#         )
#     )
    
#     # Time-based features
#     time_stats = (
#         transactions_df
#         .group_by("base_coin")
#         .agg(
#             pl.col("block_time").min().alias("first_time"),
#             pl.col("block_time").max().alias("last_time")
#         )
#         .with_columns(
#             (pl.col("last_time") - pl.col("first_time")).dt.total_seconds().alias("activity_duration_sec")
#         )
#     )
#     # Transaction velocity calculation (NEW FEATURE)
#     time_stats = time_stats.join(counts.select("base_coin", "total_txs"), on="base_coin").with_columns(
#         (pl.col("total_txs") / (pl.col("activity_duration_sec") + 1e-6)).alias("tx_per_sec")
#     )
    
#     # Slot features
#     slot_stats = (
#         transactions_df
#         .group_by("base_coin")
#         .agg(
#             pl.col("slot").min().alias("first_slot"),
#             pl.col("slot").max().alias("last_slot")
#         )
#         .with_columns(
#             (pl.col("last_slot") - pl.col("first_slot")).alias("slot_span")
#         )
#     )
    
#     # Early transactions calculation (within first 100 slots)
#     early_txs = transactions_df.group_by("base_coin").agg(
#         pl.col("tx_idx")
#         .filter(pl.col("slot") <= pl.col("slot").min() + 100)
#         .count()
#         .alias("early_txs")
#     )
    
#     # Balance features - get first and last records efficiently
#     sorted_df = transactions_df.sort(["base_coin", "slot"])
    
#     # First values
#     first_values = (
#         sorted_df
#         .group_by("base_coin")
#         .agg(
#             pl.col("virtual_token_balance_after").first().alias("first_balance"),
#             pl.col("virtual_sol_balance_after").first().alias("first_sol")
#         )
#     )
    
#     # Last values
#     last_values = (
#         sorted_df
#         .group_by("base_coin")
#         .agg(
#             pl.col("virtual_token_balance_after").last().alias("last_balance"),
#             pl.col("virtual_sol_balance_after").last().alias("last_sol")
#         )
#     )
    
#     # Combine balance features
#     balance_features = first_values.join(last_values, on="base_coin").with_columns(
#         (pl.col("last_sol") - pl.col("first_sol")).alias("sol_change")
#     )
    
#     # Balance volatility
#     balance_vol = (
#         transactions_df
#         .group_by("base_coin")
#         .agg(
#             pl.col("virtual_token_balance_after").std().alias("balance_volatility"),
#             pl.col("virtual_token_balance_after").max().alias("max_balance"),
#             pl.col("virtual_token_balance_after").min().alias("min_balance")
#         )
#     )
    
#     # Gas features
#     gas_stats = (
#         transactions_df
#         .group_by("base_coin")
#         .agg(
#             pl.col("provided_gas_fee").sum().alias("total_gas_fees"),
#             pl.col("provided_gas_fee").mean().alias("avg_gas_fee"),
#             pl.col("provided_gas_fee").std().alias("std_gas_fee"),
#             pl.col("provided_gas_fee").max().alias("max_gas_fee"),
#             pl.col("provided_gas_fee").min().alias("min_gas_fee"),
#             pl.col("provided_gas_limit").mean().alias("avg_gas_limit"),
#             pl.col("consumed_gas").mean().alias("avg_consumed_gas"),
#             pl.col("consumed_gas").sum().alias("total_consumed_gas"),
#             pl.col("fee").sum().alias("total_fee"),
#             (pl.col("provided_gas_limit") - pl.col("consumed_gas")).mean().alias("limit_utilization")
#         )
#     )

#     liq_stats = (
#     transactions_df
#       .group_by("base_coin")
#       .agg([
#         # average SOL‐balance after each transaction (pool depth proxy)
#         pl.col("virtual_sol_balance_after")
#           .mean()
#           .alias("liq_virtual_sol_balance_after_mean"),
#         # total quote‐volume (another liquidity proxy)
#         pl.col("quote_coin_amount")
#           .sum()
#           .alias("liq_quote_coin_amount"),
#       ])
#     )
#     # Gas metrics with quote
#     gas_metrics = gas_stats.join(quote_stats.select(["base_coin", "quote_sum"]), on="base_coin").with_columns([
#         (pl.col("total_fee") / (pl.col("quote_sum") + 1e-6)).alias("fee_per_sol"),
#         pl.col("avg_consumed_gas").alias("gas_per_tx"),
#         (pl.col("quote_sum") / (pl.col("total_consumed_gas") + 1e-6)).alias("gas_efficiency")
#     ])
    
#     # Wallet concentration (Gini and top wallet share) - using chunks to avoid memory issues
#     def calculate_concentration_metrics(mint):
#         mint_df = transactions_df.filter(pl.col("base_coin") == mint)
#         wallet_volumes = mint_df.group_by("signing_wallet").agg(pl.col("base_coin_amount").sum())
        
#         if wallet_volumes.height <= 1:
#             return {"base_coin": mint, "gini_coeff": 0.0, "top5_wallet_share": 0.0}
        
#         sorted_values = wallet_volumes.sort("base_coin_amount").to_pandas()["base_coin_amount"].values
#         n = len(sorted_values)
        
#         if n > 1:
#             gini_coeff = np.sum((2 * np.arange(1, n+1) - n - 1) * sorted_values) / (n * np.sum(sorted_values) + 1e-6)
#         else:
#             gini_coeff = 0.0
        
#         total_volume = sorted_values.sum()
#         if total_volume > 0:
#             top_k = min(5, n)
#             top_k_volume = np.sum(sorted_values[-top_k:])
#             top5_wallet_share = top_k_volume / total_volume
#         else:
#             top5_wallet_share = 0.0
            
#         return {"base_coin": mint, "gini_coeff": gini_coeff, "top5_wallet_share": top5_wallet_share}
    
#     print("Calculating wallet concentration metrics...")
#     unique_mints = counts["base_coin"].to_list()
    
#     # Use ThreadPoolExecutor for parallel processing
#     with concurrent.futures.ThreadPoolExecutor(max_workers=n_jobs) as executor:
#         concentration_results = list(executor.map(calculate_concentration_metrics, unique_mints))
    
#     concentration_df = pl.DataFrame(concentration_results)
    
#     # Hour distribution entropy
#     def calculate_hour_entropy(mint):
#         mint_df = transactions_df.filter(pl.col("base_coin") == mint)
#         hours = mint_df["block_time"].dt.hour()
#         hour_counts = hours.value_counts().sort("count", descending=True)
        
#         if hour_counts.height > 0:
#             total = hour_counts["count"].sum()
#             probs = hour_counts["count"] / total
#             non_zero_probs = probs.filter(probs > 0)
            
#             if non_zero_probs.len() > 0:
#                 entropy = -np.sum(non_zero_probs.to_numpy() * np.log2(non_zero_probs.to_numpy()))
#                 return {"base_coin": mint, "hour_entropy": entropy}
        
#         return {"base_coin": mint, "hour_entropy": 0.0}
    
#     print("Calculating hour entropy...")
#     with concurrent.futures.ThreadPoolExecutor(max_workers=n_jobs) as executor:
#         hour_entropy_results = list(executor.map(calculate_hour_entropy, unique_mints))
    
#     hour_entropy_df = pl.DataFrame(hour_entropy_results)
    
#     # Flow imbalance calculation
#     flow_imbalance = (
#         transactions_df
#         .group_by("base_coin")
#         .agg([
#             pl.col("quote_coin_amount").filter(pl.col("direction") == "buy").sum().alias("buy_quote_amount"),
#             pl.col("quote_coin_amount").filter(pl.col("direction") == "sell").sum().alias("sell_quote_amount")
#         ])
#         .with_columns([
#             (pl.col("buy_quote_amount") + pl.col("sell_quote_amount")).alias("total_quote_amount"),
#             ((pl.col("buy_quote_amount") - pl.col("sell_quote_amount")) / 
#              (pl.col("buy_quote_amount") + pl.col("sell_quote_amount") + 1e-6)).alias("flow_imbalance")
#         ])
#         .select(["base_coin", "flow_imbalance"])
#     )
    
#     # Price and volatility calculations
#     def calculate_price_metrics(mint):
#         mint_df = transactions_df.filter(pl.col("base_coin") == mint)
        
#         # Calculate price
#         mint_df = mint_df.with_columns(
#             (pl.col("quote_coin_amount") / pl.col("base_coin_amount")
#              .replace(0, None))
#             .alias("price")
#         )
        
#         # Replace infinite values with nulls
#         mint_df = mint_df.with_columns(
#             pl.when(pl.col("price").is_infinite())
#             .then(None)
#             .otherwise(pl.col("price"))
#             .alias("price")
#         )
        
#         avg_price = mint_df["price"].mean()
        
#         if mint_df.height >= 5:
#             # For rolling window calculations, we need to sort and convert to pandas temporarily
#             # This is a performance bottleneck, but necessary for the rolling calculation
#             sorted_df = mint_df.sort("slot").to_pandas()
#             sorted_df['rolling_price_std'] = sorted_df['price'].rolling(window=5, min_periods=1).std().fillna(0)
#             price_vol_mean = sorted_df['rolling_price_std'].mean()
#             price_vol_max = sorted_df['rolling_price_std'].max()
#         else:
#             price_vol_mean = mint_df["price"].std() if mint_df.height > 1 else 0
#             price_vol_max = price_vol_mean
        
#         price_min = mint_df["price"].min()
#         price_max = mint_df["price"].max()
#         price_mean = mint_df["price"].mean()
        
#         price_slippage = (price_max - price_min) / (price_mean + 1e-6) if price_mean is not None else 0
        
#         return {
#             "base_coin": mint,
#             "avg_price": float(avg_price) if avg_price is not None else 0.0,
#             "price_vol_mean": float(price_vol_mean) if price_vol_mean is not None else 0.0,
#             "price_vol_max": float(price_vol_max) if price_vol_max is not None else 0.0,
#             "price_slippage": float(price_slippage)
#         }
    
#     print("Calculating price and volatility metrics...")
#     with concurrent.futures.ThreadPoolExecutor(max_workers=n_jobs) as executor:
#         price_results = list(executor.map(calculate_price_metrics, unique_mints))
    
#     price_df = pl.DataFrame(price_results)
    
#     # Time-weighted features
#     def calculate_tw_features(mint):
#         mint_df = transactions_df.filter(pl.col("base_coin") == mint)
#         max_slot = mint_df["slot"].max()
        
#         # Calculate time weights using numpy instead of pl.exp
#         mint_df = mint_df.with_columns(
#             pl.col("slot").map_elements(lambda s: np.exp(-0.1 * (max_slot - s))).alias("time_weight")
#         )
        
#         tw_base = (mint_df["base_coin_amount"] * mint_df["time_weight"]).sum()
#         tw_quote = (mint_df["quote_coin_amount"] * mint_df["time_weight"]).sum()
        
#         return {
#             "base_coin": mint,
#             "tw_base_coin_amount": float(tw_base),
#             "tw_quote_coin_amount": float(tw_quote)
#         }
    
#     print("Calculating time-weighted features...")
#     with concurrent.futures.ThreadPoolExecutor(max_workers=n_jobs) as executor:
#         tw_results = list(executor.map(calculate_tw_features, unique_mints))
    
#     tw_df = pl.DataFrame(tw_results)
    
#     # Now merge all feature DataFrames
#     print("Merging all features...")
    
#     # Start with counts as the base
#     result = counts
    
#     # Add buy ratio
#     result = result.join(direction_counts, on="base_coin", how="left")
#     result = result.with_columns(
#         (pl.col("buy_count") / pl.col("total_txs")).alias("buy_ratio")
#     )
    
#     # Join remaining dataframes
#     dfs_to_join = [
#         buy_wallets, sell_wallets, base_stats, quote_stats, 
#         time_stats.select(["base_coin", "activity_duration_sec", "first_time", "last_time", "tx_per_sec"]), 
#         slot_stats, early_txs, balance_features, balance_vol, 
#         gas_metrics, concentration_df, hour_entropy_df, flow_imbalance, price_df, tw_df, liq_stats
#     ]
    
#     for df in dfs_to_join:
#         result = result.join(df, on="base_coin", how="left")
    
#     # Rename base_coin to mint for consistency with original
#     result = result.rename({"base_coin": "mint"})
    
#     # Fill null values
#     result = result.fill_null(0)
    
#     print(f"Polars processing complete with {result.height} rows and {len(result.columns)} columns")
    
#     # Convert back to pandas if needed
#     return result.to_pandas()

# # Usage:
# mint_transactions = fast_polars_process(transactions_df.iloc[:100000], n_jobs=4)

In [24]:
# import polars as pl
# print(pl.__version__)  # Should show 0.20.3 or later
# print(pl.engine)  # Should show True

In [25]:
# import polars as pl
# import numpy as np

# def optimized_polars_pipeline(transactions_df):
#     # Convert to Polars if needed
#     if not isinstance(transactions_df, pl.DataFrame):
#         transactions_df = pl.from_pandas(transactions_df)
    
#     # Ensure datetime conversion
#     if transactions_df["block_time"].dtype != pl.Datetime:
#         transactions_df = transactions_df.with_columns(
#             pl.col("block_time").str.to_datetime()
#         )
    
#     # Main processing pipeline
#     return (
#         transactions_df.lazy()
#         # Pre-sort for efficiency
#         .sort(["base_coin", "slot"])
        
#         # Base aggregations
#         .group_by("base_coin")
#         .agg([
#             # Basic counts
#             pl.count().alias("total_txs"),
#             pl.n_unique("tx_idx").alias("unique_tx_indices"),
#             pl.n_unique("signing_wallet").alias("unique_wallets"),
            
#             # Direction metrics
#             pl.col("direction").filter(pl.col("direction") == "buy").count().alias("buy_count"),
#             pl.col("direction").filter(pl.col("direction") == "sell").count().alias("sell_count"),
#             pl.col("signing_wallet").filter(pl.col("direction") == "buy").n_unique().alias("unique_buy_wallets"),
#             pl.col("signing_wallet").filter(pl.col("direction") == "sell").n_unique().alias("unique_sell_wallets"),
            
#             # Base/quote statistics
#             pl.sum("base_coin_amount").alias("base_sum"),
#             pl.mean("base_coin_amount").alias("base_mean"),
#             pl.max("base_coin_amount").alias("base_max"),
#             pl.min("base_coin_amount").alias("base_min"),
#             pl.std("base_coin_amount").alias("base_std"),
            
#             pl.sum("quote_coin_amount").alias("quote_sum"),
#             pl.mean("quote_coin_amount").alias("quote_mean"),
#             pl.max("quote_coin_amount").alias("quote_max"),
#             pl.min("quote_coin_amount").alias("quote_min"),
#             pl.std("quote_coin_amount").alias("quote_std"),
            
#             # Time features
#             pl.min("block_time").alias("first_time"),
#             pl.max("block_time").alias("last_time"),
#             (pl.max("block_time") - pl.min("block_time")).dt.seconds().alias("activity_duration_sec"),
#             pl.min("slot").alias("first_slot"),
#             pl.max("slot").alias("last_slot"),
#             (pl.max("slot") - pl.min("slot")).alias("slot_span"),
            
#             # Early transactions
#             pl.col("tx_idx").filter(pl.col("slot") <= pl.min("slot") + 100).count().alias("early_txs"),
            
#             # Balance features
#             pl.first("virtual_token_balance_after").alias("first_balance"),
#             pl.first("virtual_sol_balance_after").alias("first_sol"),
#             pl.last("virtual_token_balance_after").alias("last_balance"),
#             pl.last("virtual_sol_balance_after").alias("last_sol"),
#             pl.std("virtual_token_balance_after").alias("balance_volatility"),
#             pl.max("virtual_token_balance_after").alias("max_balance"),
#             pl.min("virtual_token_balance_after").alias("min_balance"),
            
#             # Gas features
#             pl.sum("provided_gas_fee").alias("total_gas_fees"),
#             pl.mean("provided_gas_fee").alias("avg_gas_fee"),
#             pl.std("provided_gas_fee").alias("std_gas_fee"),
#             pl.max("provided_gas_fee").alias("max_gas_fee"),
#             pl.min("provided_gas_fee").alias("min_gas_fee"),
#             pl.mean("provided_gas_limit").alias("avg_gas_limit"),
#             pl.mean("consumed_gas").alias("avg_consumed_gas"),
#             pl.sum("consumed_gas").alias("total_consumed_gas"),
#             pl.sum("fee").alias("total_fee"),
#             (pl.mean("provided_gas_limit") - pl.mean("consumed_gas")).alias("limit_utilization"),
            
#             # Liquidity proxies
#             pl.mean("virtual_sol_balance_after").alias("liq_virtual_sol_balance_after_mean"),
#         ])
        
#         # Add computed features
#         .with_columns([
#             # Transaction velocity
#             (pl.col("total_txs") / (pl.col("activity_duration_sec") + 1e-8)).alias("tx_per_sec"),
            
#             # SOL change
#             (pl.col("last_sol") - pl.col("first_sol")).alias("sol_change"),
            
#             # Buy ratio
#             (pl.col("buy_count") / pl.col("total_txs")).alias("buy_ratio"),
            
#             # Gas metrics
#             (pl.col("total_fee") / (pl.col("quote_sum") + 1e-8)).alias("fee_per_sol"),
#             (pl.col("quote_sum") / (pl.col("total_consumed_gas") + 1e-8)).alias("gas_efficiency"),
            
#             # Flow imbalance
#             ((pl.col("buy_count") - pl.col("sell_count")) / (pl.col("total_txs") + 1e-8)).alias("flow_imbalance"),
#         ])
        
#         # Add complex features through window functions
#         .with_columns([
#             # Time-weighted features
#             (pl.col("base_coin_amount") * 
#              (-0.1 * (pl.max("slot").over("base_coin") - pl.col("slot"))).exp()).sum().over("base_coin").alias("tw_base_coin_amount"),
            
#             (pl.col("quote_coin_amount") * 
#              (-0.1 * (pl.max("slot").over("base_coin") - pl.col("slot"))).exp()).sum().over("base_coin").alias("tw_quote_coin_amount"),
            
#             # Price features
#             (pl.col("quote_coin_amount") / pl.col("base_coin_amount"))
#             .fill_nan(0)
#             .alias("price")
#             .pipe(lambda s: s.rolling_std(5).over("base_coin"))
#             .mean().over("base_coin").alias("price_vol_mean"),
            
#             (pl.col("price")
#              .rolling_std(5)
#              .over("base_coin")
#              .max()
#              .alias("price_vol_max")),
            
#             ((pl.col("price").max().over("base_coin") - 
#               pl.col("price").min().over("base_coin")) / 
#              (pl.col("price").mean().over("base_coin") + 1e-8)).alias("price_slippage"),
#         ])
        
#         # Add entropy calculation
#         .with_columns(
#             (pl.col("block_time").dt.hour()
#              .value_counts()
#              .struct.field("count")
#              .entropy()
#              .over("base_coin")).alias("hour_entropy")
#         )
        
#         # Add Gini coefficient
#         .with_columns(
#             (pl.col("base_coin_amount")
#              .group_by("signing_wallet")
#              .sum()
#              .sort()
#              .pipe(lambda s: (2 * pl.arange(1, s.len() + 1) - s.len() - 1) * s)
#              .sum() / (s.sum() * s.len()))
#              .over("base_coin")
#              .alias("gini_coeff")
#         )
        
#         # Final cleanup
#         .fill_null(0)
#         .rename({"base_coin": "mint"})
#         .collect()
#         .to_pandas()
#     )

# # # Usage
# transactions_pl = pl.from_pandas(your_pandas_df)
# mint_transactions = optimized_polars_pipeline(transactions_pl)

In [26]:
mint_transactions.to_csv('mint_transactions.csv', index=False)

In [27]:
# import pandas as pd
# import numpy as np
# from joblib import Parallel, delayed


# def fast_vectorized_process(transactions_df, n_jobs=4):
#     """Vectorized version of mint processing using pandas operations"""
#     print(f"Starting vectorized transaction processing at {pd.Timestamp.now()}")
    
#     # Ensure datetime conversion
#     if not pd.api.types.is_datetime64_any_dtype(transactions_df['block_time']):
#         transactions_df['block_time'] = pd.to_datetime(transactions_df['block_time'])
    
#     # Process all mints at once with groupby operations
#     print("Calculating aggregated features...")
    
#     # Group by mint (base_coin) for bulk processing
#     grouped = transactions_df.groupby('base_coin')
    
#     # Basic counts (vectorized)
#     counts = grouped.size().reset_index(name='total_txs')
    
#     # Unique counts
#     unique_tx = grouped['tx_idx'].nunique().reset_index(name='unique_tx_indices')
#     unique_wallets = grouped['signing_wallet'].nunique().reset_index(name='unique_wallets')
    
#     # Direction-based metrics (vectorized)
#     direction_counts = transactions_df.groupby(['base_coin', 'direction']).size().unstack(fill_value=0)
#     direction_counts.columns = [f'{col}_count' for col in direction_counts.columns]
    
#     # Ensure buy and sell columns exist
#     if 'buy_count' not in direction_counts.columns:
#         direction_counts['buy_count'] = 0
#     if 'sell_count' not in direction_counts.columns:
#         direction_counts['sell_count'] = 0
    
#     # Wallet counts by direction
#     buy_wallets = transactions_df[transactions_df['direction'] == 'buy'].groupby('base_coin')['signing_wallet'].nunique()
#     sell_wallets = transactions_df[transactions_df['direction'] == 'sell'].groupby('base_coin')['signing_wallet'].nunique()
    
#     # Calculate base and quote statistics in one pass
#     base_stats = grouped['base_coin_amount'].agg(['sum', 'mean', 'max', 'min', 'std']).reset_index()
#     base_stats.columns = ['base_coin', 'base_sum', 'base_mean', 'base_max', 'base_min', 'base_std']
    
#     quote_stats = grouped['quote_coin_amount'].agg(['sum', 'mean', 'max', 'min', 'std']).reset_index()
#     quote_stats.columns = ['base_coin', 'quote_sum', 'quote_mean', 'quote_max', 'quote_min', 'quote_std']
    
#     # Time-based features
#     time_stats = grouped['block_time'].agg(['min', 'max']).reset_index()
#     time_stats.columns = ['base_coin', 'first_time', 'last_time']
#     time_stats['activity_duration_sec'] = (time_stats['last_time'] - time_stats['first_time']).dt.total_seconds()
    
#     # Slot features
#     slot_stats = grouped['slot'].agg(['min', 'max']).reset_index()
#     slot_stats.columns = ['base_coin', 'first_slot', 'last_slot'] 
#     slot_stats['slot_span'] = slot_stats['last_slot'] - slot_stats['first_slot']
    
#     # Calculate tx_per_sec safely
#     counts_with_duration = counts.merge(time_stats[['base_coin', 'activity_duration_sec']], on='base_coin')
#     counts_with_duration['tx_per_sec'] = counts_with_duration['total_txs'] / (counts_with_duration['activity_duration_sec'] + 1e-6)
    
#     # Early transactions calculation (within first 100 slots)
#     def count_early_txs(group):
#         min_slot = group['slot'].min()
#         return group[group['slot'] <= min_slot + 100]['tx_idx'].count()
    
#     early_txs = grouped.apply(count_early_txs).reset_index(name='early_txs')
    
#     # Balance features - get first and last records efficiently
#     transactions_df = transactions_df.sort_values(['base_coin', 'slot'])
#     first_idx = transactions_df.groupby('base_coin')['slot'].idxmin()
#     last_idx = transactions_df.groupby('base_coin')['slot'].idxmax()
#     first_records = transactions_df.loc[first_idx, ['base_coin', 'virtual_token_balance_after', 'virtual_sol_balance_after']]
#     last_records = transactions_df.loc[last_idx, ['base_coin', 'virtual_token_balance_after', 'virtual_sol_balance_after']]
#     first_records.columns = ['base_coin', 'first_balance', 'first_sol']
#     last_records.columns = ['base_coin', 'last_balance', 'last_sol']
#     balance_features = first_records.merge(last_records, on='base_coin')
#     balance_features['sol_change'] = balance_features['last_sol'] - balance_features['first_sol']
    
#     # Balance volatility
#     balance_vol = grouped['virtual_token_balance_after'].agg(['std', 'max', 'min']).reset_index()
#     balance_vol.columns = ['base_coin', 'balance_volatility', 'max_balance', 'min_balance']
    
#     # Gas features
#     gas_stats = grouped['provided_gas_fee'].agg(['sum', 'mean', 'std', 'max', 'min']).reset_index()
#     gas_stats.columns = ['base_coin', 'total_gas_fees', 'avg_gas_fee', 'std_gas_fee', 'max_gas_fee', 'min_gas_fee']
    
#     # Gas metrics
#     gas_metrics = pd.DataFrame({
#         'base_coin': grouped.first().index,
#         'avg_gas_limit': grouped['provided_gas_limit'].mean(),
#         'avg_consumed_gas': grouped['consumed_gas'].mean(),
#         'total_consumed_gas': grouped['consumed_gas'].sum(),
#         'total_fee': grouped['fee'].sum()
#     }).reset_index(drop=True)
#     gas_metrics['limit_utilization'] = grouped.apply(lambda x: (x['provided_gas_limit'] - x['consumed_gas']).mean()).reset_index(name='limit_utilization')['limit_utilization']
#     gas_with_quote = gas_metrics.merge(quote_stats[['base_coin', 'quote_sum']], on='base_coin')
#     gas_with_quote['fee_per_sol'] = gas_with_quote['total_fee'] / (gas_with_quote['quote_sum'] + 1e-6)
#     gas_with_quote['gas_per_tx'] = gas_with_quote['avg_consumed_gas']
#     gas_with_quote['gas_efficiency'] = gas_with_quote['quote_sum'] / (gas_with_quote['total_consumed_gas'] + 1e-6)
    
#     # Wallet concentration (Gini and top wallet share)
#     def calculate_concentration(group):
#         wallet_volumes = group.groupby('signing_wallet')['base_coin_amount'].sum()
#         if len(wallet_volumes) > 1:
#             sorted_volumes = np.sort(wallet_volumes.values)
#             n = len(sorted_volumes)
#             gini_coeff = (np.sum((2 * np.arange(1, n+1) - n - 1) * sorted_volumes) / (n * np.sum(sorted_volumes) + 1e-6))
#         else:
#             gini_coeff = 0
#         if len(wallet_volumes) > 0 and wallet_volumes.sum() > 0:
#             top5_volumes = wallet_volumes.nlargest(min(5, len(wallet_volumes)))
#             top5_wallet_share = top5_volumes.sum() / wallet_volumes.sum()
#         else:
#             top5_wallet_share = 0
#         return pd.Series({'gini_coeff': gini_coeff, 'top5_wallet_share': top5_wallet_share})
#     print("Calculating wallet concentration metrics...")
#     concentration_metrics = Parallel(n_jobs=n_jobs)(
#         delayed(lambda m: (m, calculate_concentration(transactions_df[transactions_df['base_coin'] == m])))(mint)
#         for mint in counts['base_coin'].unique()
#     )
#     concentration_df = pd.DataFrame([{'base_coin': mint, **metrics} for mint, metrics in concentration_metrics])
    
#     # Hour distribution entropy
#     def calculate_hour_entropy(group):
#         hour_counts = group['block_time'].dt.hour.value_counts()
#         hour_dist = hour_counts / hour_counts.sum() if hour_counts.sum() > 0 else hour_counts
#         non_zero_probs = hour_dist[hour_dist > 0]
#         return -np.sum(non_zero_probs * np.log2(non_zero_probs)) if len(non_zero_probs) > 0 else 0
#     hour_entropy = grouped.apply(calculate_hour_entropy).reset_index(name='hour_entropy')
    
#     # Flow imbalance calculation
#     def calculate_flow_imbalance(group):
#         buy_quote_amount = group[group['direction'] == 'buy']['quote_coin_amount'].sum()
#         sell_quote_amount = group[group['direction'] == 'sell']['quote_coin_amount'].sum()
#         total_quote_amount = buy_quote_amount + sell_quote_amount
#         return (buy_quote_amount - sell_quote_amount) / (total_quote_amount + 1e-6)
#     flow_imbalance = grouped.apply(calculate_flow_imbalance).reset_index(name='flow_imbalance')
    
#     # Price and volatility calculations
#     def calculate_price_metrics(group):
#         group = group.copy()
#         group['price'] = group['quote_coin_amount'] / group['base_coin_amount'].replace(0, np.nan)
#         group['price'] = group['price'].replace([np.inf, -np.inf], np.nan)
#         avg_price = group['price'].mean()
#         if len(group) >= 5:
#             group['rolling_price_std'] = group['price'].rolling(window=5, min_periods=1).std().fillna(0)
#             price_vol_mean = group['rolling_price_std'].mean()
#             price_vol_max = group['rolling_price_std'].max()
#         else:
#             price_vol_mean = group['price'].std()
#             price_vol_max = price_vol_mean
#         price_slippage = (group['price'].max() - group['price'].min()) / (group['price'].mean() + 1e-6)
#         return pd.Series({'avg_price': avg_price, 'price_vol_mean': price_vol_mean, 'price_vol_max': price_vol_max, 'price_slippage': price_slippage})
#     print("Calculating price and volatility metrics...")
#     price_metrics = Parallel(n_jobs=n_jobs)(
#         delayed(lambda m: (m, calculate_price_metrics(transactions_df[transactions_df['base_coin'] == m])))(mint)
#         for mint in counts['base_coin'].unique()
#     )
#     price_df = pd.DataFrame([{'base_coin': mint, **metrics} for mint, metrics in price_metrics])
    
#     # Time-weighted features
#     def calculate_tw_features(group):
#         max_slot = group['slot'].max()
#         tw_base = (group['base_coin_amount'] * np.exp(-0.1 * (max_slot - group['slot']))).sum()
#         tw_quote = (group['quote_coin_amount'] * np.exp(-0.1 * (max_slot - group['slot']))).sum()
#         return pd.Series({'tw_base_coin_amount': tw_base, 'tw_quote_coin_amount': tw_quote})
#     print("Calculating time-weighted features...")
#     tw_features = Parallel(n_jobs=n_jobs)(
#         delayed(lambda m: (m, calculate_tw_features(transactions_df[transactions_df['base_coin'] == m])))(mint)
#         for mint in counts['base_coin'].unique()
#     )
#     tw_df = pd.DataFrame([{'base_coin': mint, **metrics} for mint, metrics in tw_features])
    
#     # Now merge all feature DataFrames
#     print("Merging all features...")
#     result = counts.merge(unique_tx, on='base_coin')
#     result = result.merge(unique_wallets, on='base_coin')
#     result = result.merge(direction_counts.reset_index(), on='base_coin')
#     result['buy_ratio'] = result['buy_count'] / result['total_txs']
#     result['unique_buy_wallets'] = result['base_coin'].map(buy_wallets).fillna(0)
#     result['unique_sell_wallets'] = result['base_coin'].map(sell_wallets).fillna(0)
#     dfs_to_merge = [base_stats, quote_stats, time_stats[['base_coin', 'activity_duration_sec']], slot_stats, early_txs, balance_features, balance_vol, gas_stats, gas_with_quote, concentration_df, hour_entropy, flow_imbalance, price_df, tw_df]
#     for df in dfs_to_merge:
#         result = result.merge(df, on='base_coin', how='left')
#     result = result.rename(columns={'base_coin': 'mint'})
#     print(f"Vectorized processing complete with {len(result)} rows and {len(result.columns)} columns")
#     return result


# mint_transactions = fast_vectorized_process(transactions_df, n_jobs=4)

In [28]:
# import pandas as pd
# import numpy as np
# from joblib import Parallel, delayed
# import time
# from tqdm import tqdm
# import warnings
# warnings.filterwarnings('ignore')

# def process_mint_chunk(chunk_data, chunk_id):
#     """Process a chunk of transaction data to extract features for each mint."""
#     chunk_start = time.time()
#     unique_mints = chunk_data['base_coin'].unique()
#     print(f"[Chunk {chunk_id}] Processing {len(unique_mints)} unique mints...")
    
#     results = []
    
#     for mint in unique_mints:
#         # Get transactions for this mint
#         mint_txs = chunk_data[chunk_data['base_coin'] == mint]
        
#         # Skip if too few transactions
#         if len(mint_txs) < 3:
#             continue
        
#         # Calculate features for this mint
#         try:
#             features = calculate_mint_features(mint_txs, mint)
#             results.append(features)
#         except Exception as e:
#             print(f"[Chunk {chunk_id}] Error processing mint {mint}: {str(e)}")
    
#     chunk_duration = time.time() - chunk_start
#     print(f"[Chunk {chunk_id}] Completed in {chunk_duration:.2f} seconds, processed {len(results)}/{len(unique_mints)} mints")
#     return pd.DataFrame(results)

# def calculate_mint_features(mint_txs, mint_id):
#     """Extract all features for a single mint."""
#     # Basic counts
#     total_txs = len(mint_txs)
#     unique_tx_indices = mint_txs['tx_idx'].nunique()
#     unique_wallets = mint_txs['signing_wallet'].nunique()
    
#     # Buy/sell analysis
#     buy_txs = mint_txs[mint_txs['direction'] == 'buy']
#     sell_txs = mint_txs[mint_txs['direction'] == 'sell']
#     buy_ratio = len(buy_txs) / total_txs if total_txs > 0 else 0
#     unique_buy_wallets = buy_txs['signing_wallet'].nunique()
#     unique_sell_wallets = sell_txs['signing_wallet'].nunique()
    
#     # Base/quote amounts
#     base_sum = mint_txs['base_coin_amount'].sum()
#     base_mean = mint_txs['base_coin_amount'].mean()
#     base_max = mint_txs['base_coin_amount'].max()
#     base_min = mint_txs['base_coin_amount'].min()
#     base_std = mint_txs['base_coin_amount'].std()
    
#     quote_sum = mint_txs['quote_coin_amount'].sum()
#     quote_mean = mint_txs['quote_coin_amount'].mean()
#     quote_max = mint_txs['quote_coin_amount'].max()
#     quote_min = mint_txs['quote_coin_amount'].min()
#     quote_std = mint_txs['quote_coin_amount'].std()
    
#     # Time features
#     first_time = mint_txs['block_time'].min()
#     last_time = mint_txs['block_time'].max()
#     activity_duration_sec = (last_time - first_time).total_seconds()
    
#     first_slot = mint_txs['slot'].min()
#     last_slot = mint_txs['slot'].max()
#     slot_span = last_slot - first_slot
    
#     tx_per_sec = total_txs / (activity_duration_sec + 1e-6)
#     early_txs = mint_txs[mint_txs['slot'] <= first_slot + 100]['tx_idx'].count()
    
#     # Balance features
#     first_balance = mint_txs.iloc[0]['virtual_token_balance_after']
#     first_sol = mint_txs.iloc[0]['virtual_sol_balance_after']
#     last_balance = mint_txs.iloc[-1]['virtual_token_balance_after']
#     last_sol = mint_txs.iloc[-1]['virtual_sol_balance_after']
    
#     sol_change = last_sol - first_sol
#     balance_volatility = mint_txs['virtual_token_balance_after'].std()
#     max_balance = mint_txs['virtual_token_balance_after'].max()
#     min_balance = mint_txs['virtual_token_balance_after'].min()
    
#     # Gas features
#     total_gas_fees = mint_txs['provided_gas_fee'].sum()
#     avg_gas_fee = mint_txs['provided_gas_fee'].mean()
#     std_gas_fee = mint_txs['provided_gas_fee'].std()
#     max_gas_fee = mint_txs['provided_gas_fee'].max()
#     min_gas_fee = mint_txs['provided_gas_fee'].min()
    
#     avg_gas_limit = mint_txs['provided_gas_limit'].mean()
#     avg_consumed_gas = mint_txs['consumed_gas'].mean()
#     total_consumed_gas = mint_txs['consumed_gas'].sum()
#     total_fee = mint_txs['fee'].sum()
    
#     limit_utilization = (mint_txs['provided_gas_limit'] - mint_txs['consumed_gas']).mean()
#     fee_per_sol = total_fee / (quote_sum + 1e-6)
#     gas_per_tx = avg_consumed_gas
#     gas_efficiency = quote_sum / (total_consumed_gas + 1e-6)
    
#     # Wallet concentration
#     wallet_volumes = mint_txs.groupby('signing_wallet')['base_coin_amount'].sum()
    
#     # Gini coefficient
#     if len(wallet_volumes) > 1:
#         sorted_volumes = np.sort(wallet_volumes.values)
#         n = len(sorted_volumes)
#         gini_coeff = (np.sum((2 * np.arange(1, n+1) - n - 1) * sorted_volumes) / 
#                      (n * np.sum(sorted_volumes) + 1e-6))
#     else:
#         gini_coeff = 0
    
#     # Top wallet share
#     if len(wallet_volumes) > 0 and wallet_volumes.sum() > 0:
#         top5_volumes = wallet_volumes.nlargest(min(5, len(wallet_volumes)))
#         top5_wallet_share = top5_volumes.sum() / wallet_volumes.sum()
#     else:
#         top5_wallet_share = 0
    
#     # Flow imbalance
#     buy_quote_amount = buy_txs['quote_coin_amount'].sum()
#     sell_quote_amount = sell_txs['quote_coin_amount'].sum()
#     total_quote_amount = buy_quote_amount + sell_quote_amount
#     flow_imbalance = (buy_quote_amount - sell_quote_amount) / (total_quote_amount + 1e-6)
    
#     # Price and volatility
#     mint_txs['price'] = mint_txs['quote_coin_amount'] / mint_txs['base_coin_amount'].replace(0, np.nan)
#     mint_txs['price'] = mint_txs['price'].replace([np.inf, -np.inf], np.nan)
#     avg_price = mint_txs['price'].mean()
    
#     # Price volatility
#     if len(mint_txs) >= 5:
#         mint_txs['rolling_price_std'] = mint_txs['price'].rolling(window=5, min_periods=1).std().fillna(0)
#         price_vol_mean = mint_txs['rolling_price_std'].mean()
#         price_vol_max = mint_txs['rolling_price_std'].max()
#     else:
#         price_vol_mean = mint_txs['price'].std()
#         price_vol_max = price_vol_mean
    
#     # Slippage as liquidity indicator
#     price_slippage = (mint_txs['price'].max() - mint_txs['price'].min()) / (mint_txs['price'].mean() + 1e-6)
    
#     # Hour distribution
#     mint_txs['tx_hour'] = mint_txs['block_time'].dt.hour
#     hour_counts = mint_txs.groupby('tx_hour')['tx_idx'].count().values
#     hour_dist = hour_counts / hour_counts.sum() if hour_counts.sum() > 0 else np.zeros(len(hour_counts))
    
#     # Shannon entropy for hour distribution
#     non_zero_probs = hour_dist[hour_dist > 0]
#     hour_entropy = -np.sum(non_zero_probs * np.log2(non_zero_probs)) if len(non_zero_probs) > 0 else 0
    
#     # Time-weighted features using exponential decay
#     max_slot = mint_txs['slot'].max()
#     tw_base = (mint_txs['base_coin_amount'] * np.exp(-0.1 * (max_slot - mint_txs['slot']))).sum()
#     tw_quote = (mint_txs['quote_coin_amount'] * np.exp(-0.1 * (max_slot - mint_txs['slot']))).sum()
    
#     return {
#         'mint': mint_id,
#         'total_txs': total_txs,
#         'unique_tx_indices': unique_tx_indices,
#         'unique_wallets': unique_wallets,
#         'unique_buy_wallets': unique_buy_wallets,
#         'unique_sell_wallets': unique_sell_wallets,
#         'buy_ratio': buy_ratio,
#         'total_base': base_sum,
#         'mean_base': base_mean,
#         'max_base': base_max,
#         'min_base': base_min,
#         'std_base': base_std,
#         'total_quote': quote_sum,
#         'mean_quote': quote_mean,
#         'max_quote': quote_max,
#         'min_quote': quote_min,
#         'std_quote': quote_std,
#         'avg_price': avg_price,
#         'activity_duration_sec': activity_duration_sec,
#         'first_slot': first_slot,
#         'last_slot': last_slot,
#         'slot_span': slot_span,
#         'tx_per_sec': tx_per_sec,
#         'early_txs': early_txs,
#         'initial_balance': first_balance,
#         'initial_sol': first_sol,
#         'final_balance': last_balance,
#         'final_sol': last_sol,
#         'sol_change': sol_change,
#         'balance_volatility': balance_volatility,
#         'max_balance': max_balance,
#         'min_balance': min_balance,
#         'total_gas_fees': total_gas_fees,
#         'avg_gas_fee': avg_gas_fee,
#         'std_gas_fee': std_gas_fee,
#         'max_gas_fee': max_gas_fee,
#         'min_gas_fee': min_gas_fee,
#         'limit_utilization': limit_utilization,
#         'fee_per_sol': fee_per_sol,
#         'gas_per_tx': gas_per_tx,
#         'gas_efficiency': gas_efficiency,
#         'gini_coeff': gini_coeff,
#         'top5_wallet_share': top5_wallet_share,
#         'hour_entropy': hour_entropy,
#         'flow_imbalance': flow_imbalance,
#         'price_vol_mean': price_vol_mean,
#         'price_vol_max': price_vol_max,
#         'price_slippage': price_slippage,
#         'tw_base_coin_amount': tw_base,
#         'tw_quote_coin_amount': tw_quote
#     }

# def fast_process_mint(transactions_df, n_jobs=4, chunk_size=None):
#     """Process mint transactions in parallel chunks."""
#     total_start_time = time.time()
#     print(f"Starting transaction processing at {pd.Timestamp.now()}")
#     print(f"Total transactions: {len(transactions_df)}")
    
#     # Ensure datetime conversion is done
#     if not pd.api.types.is_datetime64_any_dtype(transactions_df['block_time']):
#         print("Converting block_time to datetime...")
#         transactions_df['block_time'] = pd.to_datetime(transactions_df['block_time'])
#         print("Datetime conversion complete")
    
#     # Sort by base_coin and slot for more efficient processing
#     print("Sorting transactions by base_coin and slot...")
#     sort_start = time.time()
#     transactions_df = transactions_df.sort_values(['base_coin', 'slot']).reset_index(drop=True)
#     print(f"Sorting complete in {time.time() - sort_start:.2f} seconds")
    
#     # Count unique mints
#     unique_mints = transactions_df['base_coin'].unique()
#     print(f"Found {len(unique_mints)} unique mints to process")
    
#     # Determine chunk size if not provided (aim for n_jobs * 5 chunks)
#     if chunk_size is None:
#         chunk_size = max(1, len(unique_mints) // (n_jobs * 5))
#     print(f"Using chunk size: {chunk_size} mints per chunk")
    
#     # Create chunks based on unique mints to avoid splitting transactions for the same mint
#     print("Creating processing chunks...")
#     chunk_start = time.time()
    
#     # Group by base_coin
#     print("Grouping by base_coin...")
#     grouped = transactions_df.groupby('base_coin')
    
#     # Create chunks of mints
#     chunks = []
#     current_chunk = []
#     current_chunk_size = 0
    
#     for mint in unique_mints:
#         current_chunk.append(mint)
#         current_chunk_size += 1
        
#         if current_chunk_size >= chunk_size:
#             chunks.append(current_chunk)
#             current_chunk = []
#             current_chunk_size = 0
    
#     # Add remaining mints as a final chunk
#     if current_chunk:
#         chunks.append(current_chunk)
    
#     print(f"Created {len(chunks)} chunks in {time.time() - chunk_start:.2f} seconds")
    
#     # Process chunks in parallel
#     print(f"Starting parallel processing with {n_jobs} jobs")
    
#     # Create a sequential version to process each chunk
#     results = []
#     # for chunk_id, chunk_mints in enumerate(chunks):
#     #     print(f"Processing chunk {chunk_id+1}/{len(chunks)} with {len(chunk_mints)} mints...")
#     #     chunk_start = time.time()
        
#     #     # Extract data for this chunk
#     #     print(f"[Chunk {chunk_id+1}] Extracting transaction data...")
#     #     extract_start = time.time()
#     #     chunk_data = pd.concat([grouped.get_group(mint) for mint in chunk_mints])
#     #     print(f"[Chunk {chunk_id+1}] Data extraction complete in {time.time() - extract_start:.2f} seconds")
        
#     #     # Process the chunk
#     #     print(f"[Chunk {chunk_id+1}] Processing features...")
#     #     process_start = time.time()
#     #     chunk_result = process_mint_chunk(chunk_data, chunk_id+1)
#     #     print(f"[Chunk {chunk_id+1}] Feature processing complete in {time.time() - process_start:.2f} seconds")
        
#     #     results.append(chunk_result)
#     #     print(f"[Chunk {chunk_id+1}] Complete in {time.time() - chunk_start:.2f} seconds, {len(chunk_result)} mints processed")
    
    
#     # Parallel version - uncomment to use joblib for parallel processing
#     print("Preparing chunk data for parallel processing...")
#     chunk_datasets = []
#     for chunk_id, chunk_mints in enumerate(chunks):
#         print(f"Preparing data for chunk {chunk_id+1}/{len(chunks)}...")
#         chunk_data = pd.concat([grouped.get_group(mint) for mint in chunk_mints])
#         chunk_datasets.append((chunk_data, chunk_id+1))
    
#     print(f"Starting parallel processing with {n_jobs} jobs...")
#     results = Parallel(n_jobs=n_jobs)(
#         delayed(process_mint_chunk)(chunk_data, chunk_id)
#         for chunk_data, chunk_id in chunk_datasets
#     )
    
    
#     # Combine results
#     print("Combining results from all chunks...")
#     combine_start = time.time()
#     if results:
#         result_df = pd.concat(results, ignore_index=True)
#         print(f"Combined {len(result_df)} mint records with {len(result_df.columns)} features in {time.time() - combine_start:.2f} seconds")
#     else:
#         print("Warning: No results to combine!")
#         result_df = pd.DataFrame()
    
#     total_duration = time.time() - total_start_time
#     print(f"Total processing completed in {total_duration:.2f} seconds ({total_duration/60:.2f} minutes)")
#     print(f"Final result: {len(result_df)} rows, {len(result_df.columns)} columns")
#     return result_df

# # Usage example - just set n_jobs and chunk_size when calling the function
# print("Starting mint processing...")
# mint_transactions = fast_process_mint(
#     transactions_df,
#     n_jobs=4,  # Adjust based on your CPU cores
#     chunk_size=100000  # Adjust based on your data size
# )

# print(f"Processing complete! Output shape: {mint_transactions.shape}")

In [29]:
# mint_transactions.shape

In [30]:
# %time
# # transactions['block_time'] = pd.to_datetime(transactions['block_time'])

# def process_group(group):
#     return pd.Series({
#         # Basic counts
#         'total_txs': group['tx_idx'].count(),
#         'unique_tx_indices': group['tx_idx'].nunique(),
#         'unique_wallets': group['signing_wallet'].nunique(),
        
#         # Base/quote amounts
#         'total_base': group['base_coin_amount'].sum(),
#         'avg_base': group['base_coin_amount'].mean(),
#         'max_base': group['base_coin_amount'].max(),
#         'min_base': group['base_coin_amount'].min(),
#         'std_base': group['base_coin_amount'].std(),
        
#         'total_quote': group['quote_coin_amount'].sum(),
#         'avg_quote': group['quote_coin_amount'].mean(),
#         'max_quote': group['quote_coin_amount'].max(),
#         'min_quote': group['quote_coin_amount'].min(),
#         'std_quote': group['quote_coin_amount'].std(),
        
#         # Direction analysis
#         'buy_ratio': (group['direction'] == 'buy').mean(),
        
#         # Time features
#         'activity_duration_sec': (
#             group['block_time'].max() - group['block_time'].min()
#         ).total_seconds(),
#         'first_slot': group['slot'].min(),
#         'last_slot': group['slot'].max(),
#         'slot_span': group['slot'].max() - group['slot'].min(),
        
#         # Balance features
#         'initial_balance': group['virtual_token_balance_after'].iloc[0] 
#             if len(group) > 0 else 0,
#         'final_balance': group['virtual_token_balance_after'].iloc[-1] 
#             if len(group) > 0 else 0,
#         'balance_volatility': group['virtual_token_balance_after'].std(),
        
#         'sol_change': (
#             group['virtual_sol_balance_after'].iloc[-1] - 
#             group['virtual_sol_balance_after'].iloc[0]
#         ) if len(group) > 0 else 0,
        
#         # Gas features
#         'total_gas_fees': group['provided_gas_fee'].sum(),
#         'avg_gas_fee': group['provided_gas_fee'].mean(),
#         'limit_utilization': (
#             group['provided_gas_limit'] - group['consumed_gas']
#         ).mean(),
#         'fee_per_sol': (
#             group['fee'].sum() / 
#             (group['quote_coin_amount'].sum() + 1e-6)
#         ),
#         'gas_per_tx': group['consumed_gas'].mean(),
#         'gas_efficiency': (
#             group['quote_coin_amount'].sum() / 
#             (group['consumed_gas'].sum() + 1e-6)
#         )
#     })

# mint_transactions = transactions.groupby('base_coin').apply(process_group).reset_index()
# mint_transactions = mint_transactions.rename(columns={'base_coin': 'mint'})

In [31]:
# import pandas as pd
# import numpy as np
# from joblib import Parallel, delayed
# import time

# start_time = time.time()

# # Make sure datetime conversion is done
# transactions['block_time'] = pd.to_datetime(transactions['block_time'])

# # Group by base_coin but don't use apply - use agg instead
# def fast_process_mint(transactions):
#     # Get unique mints
#     unique_mints = transactions['base_coin'].unique()
#     results = []
    
#     # For each computation, group once and calculate all metrics together
#     grouped = transactions.groupby('base_coin')
    
#     # Basic counts
#     counts = grouped.agg({
#         'tx_idx': ['count', 'nunique'],
#         'signing_wallet': 'nunique'
#     })
    
#     # Base/quote amounts
#     base_stats = grouped['base_coin_amount'].agg(['sum', 'mean', 'max', 'min', 'std'])
#     quote_stats = grouped['quote_coin_amount'].agg(['sum', 'mean', 'max', 'min', 'std'])
    
#     # Direction analysis
#     buy_ratio = grouped['direction'].apply(lambda x: (x == 'buy').mean())
    
#     # Time features
#     time_features = grouped.agg({
#         'block_time': [lambda x: (x.max() - x.min()).total_seconds()],
#         'slot': ['min', 'max']
#     })
#     slot_span = grouped['slot'].max() - grouped['slot'].min()
    
#     # Get first and last records for each group
#     # This is faster than using .apply on each group
#     first_idx = grouped['virtual_token_balance_after'].transform('first')
#     last_idx = grouped['virtual_token_balance_after'].transform('last')
    
#     # Create a mapping of base_coin to first/last values
#     balance_df = transactions[['base_coin', 'virtual_token_balance_after', 'virtual_sol_balance_after']].copy()
#     balance_df['is_first'] = (balance_df['virtual_token_balance_after'] == 
#                              transactions.groupby('base_coin')['virtual_token_balance_after'].transform('first'))
#     balance_df['is_last'] = (balance_df['virtual_token_balance_after'] == 
#                             transactions.groupby('base_coin')['virtual_token_balance_after'].transform('last'))
    
#     # Extract first values
#     first_balances = balance_df[balance_df['is_first']].drop_duplicates('base_coin')[['base_coin', 'virtual_token_balance_after', 'virtual_sol_balance_after']]
#     first_balances = first_balances.rename(columns={
#         'virtual_token_balance_after': 'initial_balance',
#         'virtual_sol_balance_after': 'initial_sol'
#     })
    
#     # Extract last values
#     last_balances = balance_df[balance_df['is_last']].drop_duplicates('base_coin')[['base_coin', 'virtual_token_balance_after', 'virtual_sol_balance_after']]
#     last_balances = last_balances.rename(columns={
#         'virtual_token_balance_after': 'final_balance',
#         'virtual_sol_balance_after': 'final_sol'
#     })
    
#     # Balance volatility
#     balance_vol = grouped['virtual_token_balance_after'].agg('std')
    
#     # Gas features
#     gas_features = grouped.agg({
#         'provided_gas_fee': ['sum', 'mean'],
#         'provided_gas_limit': 'mean',
#         'consumed_gas': ['mean', 'sum'],
#         'fee': 'sum'
#     })
    
#     # Calculate limit utilization
#     limit_util = grouped.apply(lambda x: (x['provided_gas_limit'] - x['consumed_gas']).mean())
    
#     # Create the final dataframe by combining all the above
#     result_df = pd.DataFrame({'mint': unique_mints})
    
#     # Add counts
#     result_df['total_txs'] = result_df['mint'].map(dict(zip(counts.index, counts[('tx_idx', 'count')])))
#     result_df['unique_tx_indices'] = result_df['mint'].map(dict(zip(counts.index, counts[('tx_idx', 'nunique')])))
#     result_df['unique_wallets'] = result_df['mint'].map(dict(zip(counts.index, counts[('signing_wallet', 'nunique')])))
    
#     # Add base/quote amounts
#     for op in ['sum', 'mean', 'max', 'min', 'std']:
#         result_df[f'total_base' if op == 'sum' else f'{op}_base'] = result_df['mint'].map(dict(zip(base_stats.index, base_stats[op])))
#         result_df[f'total_quote' if op == 'sum' else f'{op}_quote'] = result_df['mint'].map(dict(zip(quote_stats.index, quote_stats[op])))
    
#     # Add direction analysis
#     result_df['buy_ratio'] = result_df['mint'].map(dict(zip(buy_ratio.index, buy_ratio)))
    
#     # Add time features
#     result_df['activity_duration_sec'] = result_df['mint'].map(dict(zip(time_features.index, time_features[('block_time', '<lambda>')])))
#     result_df['first_slot'] = result_df['mint'].map(dict(zip(time_features.index, time_features[('slot', 'min')])))
#     result_df['last_slot'] = result_df['mint'].map(dict(zip(time_features.index, time_features[('slot', 'max')])))
#     result_df['slot_span'] = result_df['mint'].map(dict(zip(slot_span.index, slot_span)))
    
#     # Add balance features by merging
#     result_df = result_df.merge(first_balances, left_on='mint', right_on='base_coin', how='left')
#     result_df = result_df.merge(last_balances, left_on='mint', right_on='base_coin', how='left')
#     result_df = result_df.drop(['base_coin_x', 'base_coin_y'], axis=1, errors='ignore')
    
#     # Calculate sol_change
#     result_df['sol_change'] = result_df['final_sol'] - result_df['initial_sol']
#     result_df = result_df.drop(['final_sol', 'initial_sol'], axis=1)
    
#     # Add balance volatility
#     result_df['balance_volatility'] = result_df['mint'].map(dict(zip(balance_vol.index, balance_vol)))
    
#     # Add gas features
#     result_df['total_gas_fees'] = result_df['mint'].map(dict(zip(gas_features.index, gas_features[('provided_gas_fee', 'sum')])))
#     result_df['avg_gas_fee'] = result_df['mint'].map(dict(zip(gas_features.index, gas_features[('provided_gas_fee', 'mean')])))
#     result_df['limit_utilization'] = result_df['mint'].map(dict(zip(limit_util.index, limit_util)))
    
#     # Calculate fee_per_sol
#     result_df['fee_per_sol'] = result_df['mint'].map(
#         dict(zip(gas_features.index, 
#                 gas_features[('fee', 'sum')] / (quote_stats['sum'] + 1e-6)))
#     )
    
#     result_df['gas_per_tx'] = result_df['mint'].map(dict(zip(gas_features.index, gas_features[('consumed_gas', 'mean')])))
    
#     # Calculate gas_efficiency
#     result_df['gas_efficiency'] = result_df['mint'].map(
#         dict(zip(gas_features.index, 
#                 quote_stats['sum'] / (gas_features[('consumed_gas', 'sum')] + 1e-6)))
#     )
    
#     print(f"Processed {len(unique_mints)} mints in {time.time() - start_time:.2f} seconds")
    
#     return result_df

# mint_transactions = fast_process_mint(transactions)

In [32]:
mint_transactions.nunique()

mint                    1118389
total_txs                   661
unique_tx_indices           583
unique_wallets              425
buy_count                   493
                         ...   
price_vol_mean           924297
price_vol_max            685309
price_slippage           924257
tw_base_coin_amount     1042774
tw_quote_coin_amount    1034613
Length: 71, dtype: int64

In [33]:
def merge_dune_features(df, outer_df):
    outer_df = outer_df.rename(columns={'token_mint_address': 'mint'})
    outer_df = outer_df.drop_duplicates(subset='mint', keep='first')
    features_to_add = ['decimals', 'name', 'symbol', 'token_uri', 'created_at']
    
    df_merged = df.merge(
        outer_df[['mint'] + features_to_add],
        on='mint',
        how='left'
    )
    return df_merged

def merge_onchain_features(df, onchain_df):
    onchain_df = onchain_df.drop_duplicates(subset='mint', keep='first')
    df_merged = df.merge(
        onchain_df,
        on='mint',
        how='left'
    )
    return df_merged

def merge_transaction_features(df, transactions):
    transactions = transactions.drop_duplicates(subset='mint', keep='first')
    # print(f'new transactions shape : {transactions.shape}')
    df_merged = df.merge(
        transactions,
        on='mint',
        how='left'
    )
    return df_merged

train_merged = merge_dune_features(train, dune_token_df_combined)
test_merged = merge_dune_features(test, dune_token_df_combined)

train_merged_enh = merge_onchain_features(train_merged, onchain_df_2)
test_merged_enh = merge_onchain_features(test_merged, onchain_df_2)

train_merged_enh = merge_transaction_features(train_merged_enh, mint_transactions)
test_merged_enh = merge_transaction_features(test_merged_enh, mint_transactions)

In [34]:
train_merged_enh['slot_min'] = train_merged_enh['slot_min'].astype('float32')

In [35]:
from sklearn.base import TransformerMixin
import pandas as pd
import numpy as np
import time

class BlockchainPreprocessor(TransformerMixin):
    def __init__(self):
        self.hash_dims = {
            'name_x': 500,
            'symbol_x': 200,
            'token_uri': 800,
            'creator': 1000,
            'curve_address': 1500
        }
        self.numeric_clips = {}
        self.temporal_stats = {}
        self.frequency_maps = {}
        self.target_encoding = {}
        self.momentum_features = {}
        self.token_patterns = {
            'meme': r'\b(doge|shiba|floki|elon|moon|pepe|wojak|cat|dog|inu|based|frog)\b',
            'tech': r'\b(ai|token|chain|eth|sol|chain|defi|nft|swap|dao|yield|meta|web3)\b',
            'pump': r'\b(moon|pump|diamond|rocket|lambo|rich|millionaire|gold|gem)\b'
        }

    def fit(self, X, y=None):
        # Calculate temporal statistics
        temp_df = self._preprocess_time(X.copy())
        
        # Create hash features before any groupby operations
        temp_df = self._create_hash_features(temp_df)
        
        # Calculate temporal stats
        self.temporal_stats = {
            'gas_mean': temp_df['gas_used'].mean(),
            'gas_std': temp_df['gas_used'].std(),
            'gas_90th': temp_df['gas_used'].quantile(0.9),
            'dev_balance_mean': temp_df['dev_balance'].mean(),
            'dev_balance_std': temp_df['dev_balance'].std()
        }
        
        # Calculate target encoding and momentum features
        self.target_encoding['creator_grad_rate'] = temp_df.groupby('creator_hash')['has_graduated'].mean()
        graduated = temp_df[temp_df['has_graduated'] == 1]
        self.momentum_features = {
            'median_tx_velocity': graduated['tx_per_sec'].median(),
            'median_unique_wallets': graduated['unique_wallets'].median(),
            'grad_flow_imbalance': graduated['flow_imbalance'].mean()
        }
        
        # Calculate frequency maps
        self.frequency_maps['name_x_hash'] = temp_df['name_x_hash'].value_counts(normalize=True)
        self.frequency_maps['symbol_x_hash'] = temp_df['symbol_x_hash'].value_counts(normalize=True)
        
        # Network stats
        self.network_stats = {
            'flow_mean': X['flow_imbalance'].mean(),
            'flow_std': X['flow_imbalance'].std(),
            'flow_quantiles': {q: X['flow_imbalance'].quantile(q) for q in [0.25, 0.5, 0.75, 0.9]}
        }
        
        # Volatility stats
        self.volatility_stats = {
            'vol_mean': X['price_vol_mean'].mean(),
            'vol_std': X['price_vol_mean'].std(),
            'volatility_quantiles': {q: X['price_vol_mean'].quantile(q) for q in [0.25, 0.5, 0.75, 0.9]}
        }
        
        # Calculate IQR clipping ranges
        for col in ['slot_min', 'gas_used', 'dev_balance']:
            q1 = X[col].quantile(0.25)
            q3 = X[col].quantile(0.75)
            self.numeric_clips[col] = (q1 - 1.5*(q3-q1), q3 + 1.5*(q3-q1))
        
        # Creator reputation stats
        creator_stats = temp_df.groupby('creator_hash').agg({
            'has_graduated': ['mean', 'count']
        })
        creator_stats.columns = ['_'.join(col).strip() for col in creator_stats.columns.values]
        self.creator_stats = creator_stats
        
        return self

    def transform(self, X):
        df = X.copy()
        
        df = self._preprocess_time(df)
        df = self._handle_names(df)
        df = self._parse_bundle(df)
        df = self._create_hash_features(df)
        df = self._process_numerics(df)
        df = self._create_frequency_features(df)
        
        df = self._create_temporal_features(df)
        df = self._create_interaction_features(df)
        df = self._create_anomaly_features(df)
        df = self._create_text_features(df)
        df = self._create_target_encoding(df)
        df = self._process_network_features(df)
        df = self._process_volatility_features(df)
        df = self._create_liquidity_indicators(df)
        df = self._create_momentum_features(df)
        df = self._create_token_type_features(df)
        df = self._create_creator_reputation_features(df)
        df = self._create_advanced_liquidity_features(df)
        df = self._create_whale_activity_features(df)
        df = self._create_datetime_features_pandas(df)
        
        return self._final_cleanup(df)

    def _preprocess_time(self, df):
        time_cols = ['created_at', 'block_time']
        for col in time_cols:
            df[col] = pd.to_datetime(df[col], utc=True).dt.tz_convert(None)

        df['coin_age'] = (df['block_time'] - df['created_at']).dt.total_seconds()
        df['peak_launch'] = df['created_at'].dt.hour.between(9, 17).astype(int)
        
        for prefix in ['created', 'block']:
            dt_col = f'{prefix}_at' if prefix == 'created' else f'{prefix}_time'
            for unit in ['day', 'hour', 'month']:
                col = f'{prefix}_{unit}'
                df[col] = getattr(df[dt_col].dt, unit)
                df[f'{col}_sin'] = np.sin(2 * np.pi * df[col] / self._get_period(unit))
                df[f'{col}_cos'] = np.cos(2 * np.pi * df[col] / self._get_period(unit))
        
        return df

    def _get_period(self, unit):
        return {'day': 31, 'hour': 24, 'month': 12}.get(unit, 1)

    def _create_temporal_features(self, df):
        df['tx_velocity'] = df['tx_idx'] / (df['coin_age'] + 1e-6)
        df['block_saturation'] = df.groupby('slot')['tx_idx'].transform(lambda x: x / x.max())
        return df

    def _create_interaction_features(self, df):
        action_cols = ['nft_actions', 'swap_actions', 'other_actions']
        for col in action_cols:
            if col not in df.columns:
                df[col] = 0
        
        df['total_actions'] = df[action_cols].sum(axis=1)
        df['swap_action_ratio'] = df['swap_actions'] / (df['total_actions'].replace(0, 1))
        df['gas_per_instruction'] = df['gas_used'] / (df['amount_of_instructions'].replace(0, 1))
        return df

    def _create_anomaly_features(self, df):
        df['dev_balance_z'] = (df['dev_balance'] - self.temporal_stats['dev_balance_mean']) / self.temporal_stats['dev_balance_std']
        df['gas_z'] = (df['gas_used'] - self.temporal_stats['gas_mean']) / self.temporal_stats['gas_std']
        df['gas_outlier'] = (df['gas_used'] > self.temporal_stats['gas_90th']).astype(int)
        return df

    def _create_text_features(self, df):
        df['name'] = df['name_x'].fillna(df.get('name_y', ''))
        df['name_length'] = df['name'].str.len().fillna(0)
        df['has_meme_keyword'] = df['name'].str.contains(r'\b(doge|shiba|floki|elon|moon)\b', case=False, na=False).astype(int)
        return df

    def _create_target_encoding(self, df):
        df['creator_grad_rate'] = df['creator_hash'].map(self.target_encoding['creator_grad_rate']).fillna(df['has_graduated'].mean())
        return df

    def _handle_names(self, df):
        df['name'] = df['name_x'].fillna(df.get('name_y', ''))
        df['symbol'] = df['symbol_x'].fillna(df.get('symbol_y', ''))
        return df

    def _parse_bundle(self, df):
        def parse_bundle(s):
            try:
                return {
                    'nft_actions': s.count('NFT_MINT'),
                    'swap_actions': s.count('TOKEN_SWAP'),
                    'other_actions': s.count('UNRECOGNIZED_ACTION')
                }
            except:
                return {'nft_actions': 0, 'swap_actions': 0, 'other_actions': 0}
        
        bundle_data = df['bundle_structure'].fillna('').apply(parse_bundle)
        action_counts = pd.json_normalize(bundle_data)
        df = pd.concat([df, action_counts], axis=1)
        
        for col in ['nft_actions', 'swap_actions', 'other_actions']:
            if col not in df.columns:
                df[col] = 0
        return df

    def _create_hash_features(self, df):
        for col, dim in self.hash_dims.items():
            df[f'{col}_hash'] = (df[col].fillna('missing').astype(str).apply(lambda x: (hash(x) % dim) + 1))
        return df

    def _process_numerics(self, df):
        for col in ['gas_used', 'dev_balance']:
            clip_min, clip_max = self.numeric_clips[col]
            df[col] = (df[col].fillna(df[col].median()).clip(lower=clip_min, upper=clip_max).pipe(np.log1p))
        
        df['complexity_ratio'] = df['amount_of_instructions'] / df['bundle_size'].replace(0, 1e-6)
        return df

    def _create_frequency_features(self, df):
        for col in ['pf_program_index', 'creation_ix_index']:
            freq = df[col].value_counts(normalize=True).add(1e-6).to_dict()
            df[f'{col}_freq'] = df[col].map(freq).fillna(1e-6)
        return df

    def _process_network_features(self, df):
        df['flow_zscore'] = (df['flow_imbalance'] - self.network_stats['flow_mean']) / self.network_stats['flow_std']
        return df

    def _process_volatility_features(self, df):
        df['volatility_ratio'] = df['price_vol_mean'] / (df['avg_price'] + 1e-6)
        df['vol_zscore'] = (df['price_vol_mean'] - self.volatility_stats['vol_mean']) / self.volatility_stats['vol_std']
        return df

    def _create_liquidity_indicators(self, df):
        df['depth_ratio'] = df['quote_sum'] / (df['liq_virtual_sol_balance_after_mean'] + 1e-6)
        df['slippage_ratio'] = df['liq_quote_coin_amount'] / (df['quote_sum'] + 1e-6)
        return df

    def _process_transaction_features(self, df):
        df['whale_impact'] = df['top5_wallet_share'] * df['total_base']
        df['volatility_intensity'] = df['tx_per_sec'] * df['price_vol_mean']
        df['concentration_risk'] = df['gini_coeff'] * np.log1p(df['unique_wallets'])
        return df

    def _create_momentum_features(self, df):
        df['velocity_vs_grads'] = df['tx_per_sec'] / (self.momentum_features['median_tx_velocity'] + 1e-6)
        df['wallet_adoption_vs_grads'] = df['unique_wallets'] / (self.momentum_features['median_unique_wallets'] + 1e-6)
        df['wallet_growth_rate'] = df['unique_wallets'] / (df['activity_duration_sec']/3600 + 1e-6)
        df['flow_vs_grads'] = df['flow_imbalance'] / (self.momentum_features['grad_flow_imbalance'] + 1e-6)
        df['early_adoption_ratio'] = df['early_txs'] / (df['total_txs'] + 1e-6)
        return df

    def _create_token_type_features(self, df):
        for pattern_name, regex in self.token_patterns.items():
            df[f'is_{pattern_name}_token'] = df['name'].str.lower().str.contains(regex, case=False, na=False).astype(int)
        df['name_uniqueness'] = df['name_length'] * (1 - df['name_x_hash'].map(self.frequency_maps.get('name_x_hash', pd.Series())).fillna(0.5))
        return df

    def _create_creator_reputation_features(self, df):
        for feature in ['has_graduated_mean', 'has_graduated_count']:
            df[feature] = df['creator_hash'].map(self.creator_stats[feature])
        prior_mean = df['has_graduated'].mean()
        prior_count = 2
        df['creator_reputation'] = (
            (df['has_graduated_mean'] * df['has_graduated_count'] + prior_mean * prior_count) / 
            (df['has_graduated_count'] + prior_count)
        )
        df['creator_experience'] = np.log1p(df['has_graduated_count'])
        return df

    def _create_advanced_liquidity_features(self, df):
        df['liquidity_strength'] = df['quote_sum'] / (df['price_slippage'] + 1e-6)
        df['liquidity_stability'] = 1 / (df['price_vol_mean'] * df['top5_wallet_share'] + 1e-6)
        df['buy_pressure'] = df['buy_ratio'] * df['flow_imbalance']
        df['liquidity_per_wallet'] = df['quote_sum'] / (df['unique_wallets'] + 1e-6)
        return df

    def _create_whale_activity_features(self, df):
        df['whale_dominance'] = df['top5_wallet_share'] * np.sqrt(df['unique_wallets'])
        df['whale_manipulation'] = df['gini_coeff'] * df['price_vol_mean']
        df['whale_balance_ratio'] = df['max_balance'] / (df['base_sum'] + 1e-6)
        return df

    def _create_datetime_features_pandas(self, df):

        df = df.copy()
        df['first_time'] = pd.to_datetime(df['first_time'])
        df['last_time'] = pd.to_datetime(df['last_time'])
    
    # 1. Duration Features
        df['duration_sec'] = (df['last_time'] - df['first_time']).dt.total_seconds()
    
    # # Binned durations - use right=False to make intervals left-inclusive
    #     bins = [0, 1, 5, 30, 60, 300, np.inf]
    #     labels = ['<1s', '1-5s', '5-30s', '30-60s', '1-5m', '5m+']
    #     df['duration_bin'] = pd.cut(df['duration_sec'], bins=bins, labels=labels, right=False)
    
    # 2. Temporal Intensity
        df['tx_intensity'] = df['total_txs'] / (df['duration_sec'] + 1e-6)  # Transactions per second
    
    # 3. Time-of-Day Features (Cyclical Encoding)
        df['hour_sin_f'] = np.sin(2 * np.pi * df['first_time'].dt.hour / 24)
        df['hour_cos_f'] = np.cos(2 * np.pi * df['first_time'].dt.hour / 24)
        df['minute_sin_f'] = np.sin(2 * np.pi * df['first_time'].dt.minute / 60)
        df['hour_sin_f'] = np.sin(2 * np.pi * df['last_time'].dt.hour / 24)
        df['hour_cos_f'] = np.cos(2 * np.pi * df['last_time'].dt.hour / 24)
        df['minute_sin_f'] = np.sin(2 * np.pi * df['last_time'].dt.minute / 60)
    
    # 6. Burst Detection
        df['burstiness'] = 1 - (df['duration_sec'] / 
                          (df['last_time'].max() - df['first_time'].min()).total_seconds())

        # df['time_since_last'] = df.groupby('mint')['first_time'].diff().dt.total_seconds()
    
    # 8. Is Opening Activity (first transaction of the day)
        # df['is_opening_activity'] = (
        #     df['first_time'].dt.normalize() == 
        #     df.groupby('mint')['first_time'].transform('min').dt.normalize()
        # ).astype(int)
    
        return df

    def _final_cleanup(self, df):
        cols_to_drop = [
            'mint', 'slot_graduated', 'has_graduated',
            'name_x', 'symbol_x', 'token_uri', 'creator',
            'curve_address', 'bundle_structure', 'url', 'version',
            'first_time', 'last_time'
        ]
        existing_cols = [col for col in cols_to_drop if col in df.columns]
        return df.drop(columns=existing_cols, errors='ignore')

# Example usage
test_merged_enh['slot_graduated'] = np.nan
test_merged_enh['has_graduated'] = np.nan

start_time = time.time()
preprocessor = BlockchainPreprocessor()
train_clean = preprocessor.fit_transform(train_merged_enh)
test_clean = preprocessor.transform(test_merged_enh)
print(f'processed both train and test set in: {time.time() - start_time:.2f} seconds')

/usr/local/lib/python3.10/dist-packages/pandas/core/arraylike.py:399: RuntimeWarning: invalid value encountered in sin
  result = getattr(ufunc, method)(*inputs, **kwargs)
/usr/local/lib/python3.10/dist-packages/pandas/core/arraylike.py:399: RuntimeWarning: invalid value encountered in cos
  result = getattr(ufunc, method)(*inputs, **kwargs)
/usr/local/lib/python3.10/dist-packages/pandas/core/arraylike.py:399: RuntimeWarning: invalid value encountered in sin
  result = getattr(ufunc, method)(*inputs, **kwargs)
/usr/local/lib/python3.10/dist-packages/pandas/core/arraylike.py:399: RuntimeWarning: invalid value encountered in cos
  result = getattr(ufunc, method)(*inputs, **kwargs)
/usr/local/lib/python3.10/dist-packages/pandas/core/arraylike.py:399: RuntimeWarning: invalid value encountered in sin
  result = getattr(ufunc, method)(*inputs, **kwargs)
/usr/local/lib/python3.10/dist-packages/pandas/core/arraylike.py:399: RuntimeWarning: invalid value encountered in cos
  result = getattr(uf

processed both train and test set in: 270.64 seconds


In [36]:
# train_clean.head(3)

In [37]:
# from sklearn.base import TransformerMixin
# import pandas as pd
# import numpy as np
# import time

# class BlockchainPreprocessor(TransformerMixin):
#     def __init__(self):
#         self.hash_dims = {
#             'name_x': 500,
#             'symbol_x': 200,
#             'token_uri': 800,
#             'creator': 1000,
#             'curve_address': 1500
#         }
#         self.numeric_clips = {}
#         self.temporal_stats = {}
#         self.frequency_maps = {}
#         self.target_encoding = {}
#         self.momentum_features = {}
#         self.token_patterns = {
#             'meme': r'\b(doge|shiba|floki|elon|moon|pepe|wojak|cat|dog|inu|based|frog)\b',
#             'tech': r'\b(ai|token|chain|eth|sol|chain|defi|nft|swap|dao|yield|meta|web3)\b',
#             'pump': r'\b(moon|pump|diamond|rocket|lambo|rich|millionaire|gold|gem)\b'
#         }

#     def fit(self, X, y=None):
#         # Calculate temporal statistics
#         temp_df = self._preprocess_time(X.copy())
        
#         # Create hash features before any groupby operations
#         temp_df = self._create_hash_features(temp_df)
        
#         # Calculate temporal stats
#         self.temporal_stats = {
#             'gas_mean': temp_df['gas_used'].mean(),
#             'gas_std': temp_df['gas_used'].std(),
#             'gas_90th': temp_df['gas_used'].quantile(0.9),
#             'dev_balance_mean': temp_df['dev_balance'].mean(),
#             'dev_balance_std': temp_df['dev_balance'].std()
#         }
        
#         # Calculate target encoding after creating hashes
#         if 'has_graduated' in temp_df.columns:
#             self.target_encoding['creator_grad_rate'] = temp_df.groupby('creator_hash')['has_graduated'].mean()
            
#             # NEW: Extract momentum features if target exists
#             graduated = temp_df[temp_df['has_graduated'] == 1]
#             self.momentum_features = {
#                 'median_tx_velocity': graduated['tx_per_sec'].median(),
#                 'median_unique_wallets': graduated['unique_wallets'].median(),
#                 'grad_flow_imbalance': graduated['flow_imbalance'].mean() if 'flow_imbalance' in graduated.columns else 0
#             }
        
#         # Calculate frequency maps after creating hashes
#         self.frequency_maps['name_x_hash'] = temp_df['name_x_hash'].value_counts(normalize=True)
#         self.frequency_maps['symbol_x_hash'] = temp_df['symbol_x_hash'].value_counts(normalize=True)
        
#         if 'flow_imbalance' in X.columns:
#             self.network_stats = {
#                 'flow_mean': X['flow_imbalance'].mean(),
#                 'flow_std': X['flow_imbalance'].std(),
#                 # NEW: Temporal flow patterns
#                 'flow_quantiles': {q: X['flow_imbalance'].quantile(q) for q in [0.25, 0.5, 0.75, 0.9]}
#             }
            
#         # Volatility normalization
#         if 'price_vol_mean' in X.columns:
#             self.volatility_stats = {
#                 'vol_mean': X['price_vol_mean'].mean(),
#                 'vol_std': X['price_vol_mean'].std(),
#                 # NEW: Market sentiment indicators
#                 'volatility_quantiles': {q: X['price_vol_mean'].quantile(q) for q in [0.25, 0.5, 0.75, 0.9]}
#             }
            
#         # Calculate IQR clipping ranges
#         for col in ['slot_min', 'gas_used', 'dev_balance']:
#             if col in X.columns:
#                 q1 = X[col].quantile(0.25)
#                 q3 = X[col].quantile(0.75)
#                 self.numeric_clips[col] = (q1 - 1.5*(q3-q1), q3 + 1.5*(q3-q1))
        
#         # NEW: Creator reputation stats
#         if 'creator' in X.columns:
#             creator_stats = temp_df.groupby('creator_hash').agg({
#                 'has_graduated': ['mean', 'count']
#             })
#             creator_stats.columns = ['_'.join(col).strip() for col in creator_stats.columns.values]
#             self.creator_stats = creator_stats
        
#         return self

#     def transform(self, X):
#         df = X.copy()
        
#         df = self._preprocess_time(df)
#         df = self._handle_names(df)
#         df = self._parse_bundle(df)
#         df = self._create_hash_features(df)
#         df = self._process_numerics(df)
#         df = self._create_frequency_features(df)
        
#         df = self._create_temporal_features(df)
#         df = self._create_interaction_features(df)
#         df = self._create_anomaly_features(df)
#         df = self._create_text_features(df)
#         df = self._create_target_encoding(df)
#         df = self._process_network_features(df)
#         df = self._process_volatility_features(df)
#         df = self._create_liquidity_indicators(df)
        
#         # NEW FEATURES
#         df = self._create_momentum_features(df)
#         df = self._create_token_type_features(df)
#         df = self._create_creator_reputation_features(df)
#         df = self._create_advanced_liquidity_features(df)
#         df = self._create_whale_activity_features(df)
        
#         return self._final_cleanup(df)

#     def _preprocess_time(self, df):
#         time_cols = ['created_at', 'block_time']
#         for col in time_cols:
#             if col in df.columns:
#                 df[col] = pd.to_datetime(df[col], utc=True).dt.tz_convert(None)

#         df['coin_age'] = (df['block_time'] - df['created_at']).dt.total_seconds()
#         df['peak_launch'] = df['created_at'].dt.hour.between(9, 17).astype(int)
        
#         for prefix in ['created', 'block']:
#             dt_col = f'{prefix}_at' if prefix == 'created' else f'{prefix}_time'
#             if dt_col in df.columns:
#                 for unit in ['day', 'hour', 'month']:
#                     col = f'{prefix}_{unit}'
#                     df[col] = getattr(df[dt_col].dt, unit)
#                     df[f'{col}_sin'] = np.sin(2 * np.pi * df[col] / self._get_period(unit))
#                     df[f'{col}_cos'] = np.cos(2 * np.pi * df[col] / self._get_period(unit))
        
#         return df

#     def _get_period(self, unit):
#         return {
#             'day': 31,
#             'hour': 24,
#             'month': 12
#         }.get(unit, 1)

#     def _create_temporal_features(self, df):
#         # Transaction velocity
#         df['tx_velocity'] = df['tx_idx'] / (df['coin_age'] + 1e-6)
        
#         # Block saturation
#         df['block_saturation'] = df.groupby('slot')['tx_idx'].transform(
#             lambda x: x / x.max()
#         )
#         return df

#     def _create_interaction_features(self, df):
#         """Create interaction features with safety checks"""
#         # Verify action columns exist
#         action_cols = ['nft_actions', 'swap_actions', 'other_actions']
#         for col in action_cols:
#             if col not in df.columns:
#                 df[col] = 0
    
#         # Create interaction features
#         df['total_actions'] = df[action_cols].sum(axis=1)
#         df['swap_action_ratio'] = df['swap_actions'] / (df['total_actions'].replace(0, 1))  # Prevent div/0
    
#         # Gas efficiency metrics
#         if 'amount_of_instructions' in df.columns:
#             df['gas_per_instruction'] = df['gas_used'] / (df['amount_of_instructions'].replace(0, 1))
    
#         return df

#     def _create_anomaly_features(self, df):
#         # Z-scores
#         df['dev_balance_z'] = (df['dev_balance'] - self.temporal_stats['dev_balance_mean']) / self.temporal_stats['dev_balance_std']
#         df['gas_z'] = (df['gas_used'] - self.temporal_stats['gas_mean']) / self.temporal_stats['gas_std']
        
#         # Outlier flags
#         df['gas_outlier'] = (df['gas_used'] > self.temporal_stats['gas_90th']).astype(int)
#         return df

#     def _create_text_features(self, df):
#         # Name analysis
#         df['name'] = df['name_x'].fillna(df.get('name_y', ''))
#         df['name_length'] = df['name'].str.len().fillna(0)
#         df['has_meme_keyword'] = df['name'].str.contains(
#             r'\b(doge|shiba|floki|elon|moon)\b', 
#             case=False, 
#             na=False
#         ).astype(int)
#         return df

#     def _create_target_encoding(self, df):
#         if self.target_encoding.get('creator_grad_rate') is not None:
#             df['creator_grad_rate'] = df['creator_hash'].map(
#                 self.target_encoding['creator_grad_rate']
#             ).fillna(df['has_graduated'].mean())
#         return df

#     # Existing helper methods (updated with safety checks)
#     def _handle_names(self, df):
#         df['name'] = df['name_x'].fillna(df.get('name_y', ''))
#         df['symbol'] = df['symbol_x'].fillna(df.get('symbol_y', ''))
#         return df

#     def _parse_bundle(self, df):
#         if 'bundle_structure' in df.columns:
#         # Parse bundle structure and create raw action columns
#             def parse_bundle(s):
#                 try:
#                     return {
#                         'nft_actions': s.count('NFT_MINT'),
#                         'swap_actions': s.count('TOKEN_SWAP'),
#                         'other_actions': s.count('UNRECOGNIZED_ACTION')
#                         }
#                 except:
#                     return {'nft_actions': 0, 'swap_actions': 0, 'other_actions': 0}
        
#             bundle_data = df['bundle_structure'].fillna('').apply(parse_bundle)
#             action_counts = pd.json_normalize(bundle_data)
#             df = pd.concat([df, action_counts], axis=1)
        
#         # Ensure action columns exist even if parsing fails
#         for col in ['nft_actions', 'swap_actions', 'other_actions']:
#             if col not in df.columns:
#                 df[col] = 0
            
#         return df

#     def _create_hash_features(self, df):
#     # Create hash-based features with safety checks
#         for col, dim in self.hash_dims.items():
#             if col in df.columns:
#             # Handle missing values and ensure string type
#                 df[f'{col}_hash'] = (
#                     df[col]
#                     .fillna('missing')
#                     .astype(str)
#                     .apply(lambda x: (hash(x) % dim) + 1)  # +1 to avoid 0
#                 )
#             else:
#             # Create default hash if column missing
#                 df[f'{col}_hash'] = 1
#         return df

#     def _process_numerics(self, df):
#         # Process numerical features with robust error handling
#         for col in ['gas_used', 'dev_balance']:
#             if col in df.columns and col in self.numeric_clips:
#                 # Apply safe clipping with fallbacks
#                 clip_min, clip_max = self.numeric_clips[col]
#                 df[col] = (
#                     df[col]
#                     .fillna(df[col].median())
#                     .clip(lower=clip_min, upper=clip_max)
#                     .pipe(np.log1p)
#                 )
        
#         # Add complexity ratio with division guard
#         if 'amount_of_instructions' in df.columns and 'bundle_size' in df.columns:
#             df['complexity_ratio'] = (
#                 df['amount_of_instructions'] / 
#                 df['bundle_size'].replace(0, 1e-6)
#             )
#         return df

#     def _create_frequency_features(self, df):
#         # Create frequency-encoded features with smoothing
#         for col in ['pf_program_index', 'creation_ix_index']:
#             if col in df.columns:
#                 # Calculate frequencies with additive smoothing
#                 freq = (
#                     df[col].value_counts(normalize=True)
#                     .add(1e-6)  # Prevent zero probabilities
#                     .to_dict()
#                 )
#                 df[f'{col}_freq'] = df[col].map(freq).fillna(1e-6)
#         return df

#     def _process_network_features(self, df):
#         if 'flow_imbalance' in df.columns:
#             df['flow_zscore'] = (df['flow_imbalance'] - self.network_stats['flow_mean']) / \
#                                self.network_stats['flow_std']
#         return df

#     def _process_volatility_features(self, df):
#         if 'price_vol_mean' in df.columns:
#             df['volatility_ratio'] = df['price_vol_mean'] / (df['avg_price'] + 1e-6)
#             df['vol_zscore'] = (df['price_vol_mean'] - self.volatility_stats['vol_mean']) / \
#                              self.volatility_stats['vol_std']
#         return df

#     def _create_liquidity_indicators(self, df):
#         if 'liq_virtual_sol_balance_after_mean' in df.columns:
#             df['depth_ratio'] = df['total_quote'] / (df['liq_virtual_sol_balance_after_mean'] + 1e-6)
#             df['slippage_ratio'] = df['liq_quote_coin_amount'] / (df['total_quote'] + 1e-6)
#         return df

#     def _process_transaction_features(self, df):
#         # Enhanced version with new ratios
#         if 'total_txs' in df.columns and 'unique_wallets' in df.columns:
#             df['whale_impact'] = df['top5_wallet_share'] * df['total_base']
            
#         if 'tx_per_sec' in df.columns and 'price_vol_mean' in df.columns:
#             df['volatility_intensity'] = df['tx_per_sec'] * df['price_vol_mean']
            
#         # Add interaction terms
#         if 'gini_coeff' in df.columns and 'unique_wallets' in df.columns:
#             df['concentration_risk'] = df['gini_coeff'] * np.log1p(df['unique_wallets'])
            
#         return df

#     def _create_momentum_features(self, df):
#         """Create momentum and velocity-based features that indicate token adoption rate"""
#         if not self.momentum_features:
#             return df
            
#         # Velocity ratio compared to graduated tokens
#         if 'tx_per_sec' in df.columns:
#             df['velocity_vs_grads'] = df['tx_per_sec'] / (self.momentum_features['median_tx_velocity'] + 1e-6)
            
#         # Wallet adoption compared to graduated tokens
#         if 'unique_wallets' in df.columns:
#             df['wallet_adoption_vs_grads'] = df['unique_wallets'] / (self.momentum_features['median_unique_wallets'] + 1e-6)
            
#         # Calculate adoption acceleration (wallet growth over time)
#         if 'unique_wallets' in df.columns and 'activity_duration_sec' in df.columns:
#             df['wallet_growth_rate'] = df['unique_wallets'] / (df['activity_duration_sec']/3600 + 1e-6)  # per hour
            
#         # Flow momentum vs graduated tokens
#         if 'flow_imbalance' in df.columns:
#             df['flow_vs_grads'] = df['flow_imbalance'] / (self.momentum_features['grad_flow_imbalance'] + 1e-6)
            
#         # Early adoption momentum
#         if 'early_txs' in df.columns and 'total_txs' in df.columns:
#             df['early_adoption_ratio'] = df['early_txs'] / (df['total_txs'] + 1e-6)
            
#         return df
        
#     def _create_token_type_features(self, df):
#         """Classify tokens based on naming patterns that correlate with graduation"""
#         if 'name' not in df.columns:
#             return df
            
#         # Token type detection based on name patterns
#         for pattern_name, regex in self.token_patterns.items():
#             df[f'is_{pattern_name}_token'] = df['name'].str.lower().str.contains(
#                 regex, case=False, na=False
#             ).astype(int)
            
#         # Calculate name uniqueness score
#         if 'name_length' in df.columns:
#             # Unique tokens tend to have longer, more distinct names
#             df['name_uniqueness'] = df['name_length'] * (1 - df['name_x_hash'].map(
#                 self.frequency_maps.get('name_x_hash', pd.Series())
#             ).fillna(0.5))
            
#         return df
        
#     def _create_creator_reputation_features(self, df):
#         """Add features capturing creator's historical success with tokens"""
#         if not hasattr(self, 'creator_stats') or 'creator_hash' not in df.columns:
#             return df
            
#         # Join creator historical performance
#         creator_features = ['has_graduated_mean', 'has_graduated_count']
        
#         for feature in creator_features:
#             if feature in self.creator_stats.columns:
#                 df[feature] = df['creator_hash'].map(self.creator_stats[feature])
                
#         # Calculate reputation scores
#         if 'has_graduated_mean' in df.columns and 'has_graduated_count' in df.columns:
#             # Bayesian average with pseudocounts for reliability
#             prior_mean = df['has_graduated'].mean() if 'has_graduated' in df.columns else 0.5
#             prior_count = 2  # Prior strength
            
#             df['creator_reputation'] = (
#                 (df['has_graduated_mean'] * df['has_graduated_count'] + prior_mean * prior_count) / 
#                 (df['has_graduated_count'] + prior_count)
#             )
            
#             # Experience factor - creators with more tokens have more experience
#             df['creator_experience'] = np.log1p(df['has_graduated_count'])
            
#         return df
        
#     def _create_advanced_liquidity_features(self, df):
#         """Create features that better characterize liquidity patterns"""
#         liquidity_cols = [col for col in df.columns if 'liq_' in col or 'slippage' in col]
#         if not liquidity_cols:
#             return df
            
#         # Liquidity depth measures
#         # if 'quote_sum' in df.columns and 'price_slippage' in df.columns:
#             df['liquidity_strength'] = df['quote_sum'] / (df['price_slippage'] + 1e-6)
            
#         # Liquidity stability - how consistent the pools are
#         # if 'price_vol_mean' in df.columns and 'top5_wallet_share' in df.columns:
#             df['liquidity_stability'] = 1 / (df['price_vol_mean'] * df['top5_wallet_share'] + 1e-6)
            
#         # Liquidity imbalance - buy vs sell pressure
#         # if 'buy_ratio' in df.columns and 'flow_imbalance' in df.columns:
#             df['buy_pressure'] = df['buy_ratio'] * df['flow_imbalance']
            
#         # Liquidity saturation - how much the market is able to absorb
#         # if 'total_quote' in df.columns and 'unique_wallets' in df.columns:
#             df['liquidity_per_wallet'] = df['total_quote'] / (df['unique_wallets'] + 1e-6)
            
#         return df
        
#     def _create_whale_activity_features(self, df):
#         # """Create features to capture large holder (whale) behavior"""
#         # if 'top5_wallet_share' not in df.columns:
#             # return df
            
#         # Concentration risk factor 
#         # if 'unique_wallets' in df.columns:
#             df['whale_dominance'] = df['top5_wallet_share'] * np.sqrt(df['unique_wallets'])
            
#         # Whale activity detection
#         # if 'gini_coeff' in df.columns and 'price_vol_mean' in df.columns:
#             df['whale_manipulation'] = df['gini_coeff'] * df['price_vol_mean']
            
#         # Whale balance indicators
#         # if 'max_balance' in df.columns and 'total_base' in df.columns:
#             df['whale_balance_ratio'] = df['max_balance'] / (df['base_sum'] + 1e-6)
            
#             return df

#     def _final_cleanup(self, df):
#         # Safe column removal with existence checks
#         cols_to_drop = [
#             'mint', 'slot_graduated', 'has_graduated',
#             'name_x', 'symbol_x', 'token_uri', 'creator',
#             'curve_address', 'bundle_structure', 'url', 'version'
#         ]
#         existing_cols = [col for col in cols_to_drop if col in df.columns]
#         return df.drop(columns=existing_cols, errors='ignore')

# test_merged_enh['slot_graduated'] = np.nan
# test_merged_enh['has_graduated'] = np.nan

# start_time = time.time()
# preprocessor = BlockchainPreprocessor()
# train_clean = preprocessor.fit_transform(train_merged_enh.iloc[:1000])
# test_clean = preprocessor.transform(test_merged_enh.iloc[:1000])
# print(f'processed both train and test set in: {time.time() - start_time:.2f} seconds')

In [38]:
# train_clean.shape

In [39]:
# from sklearn.compose import ColumnTransformer
# from sklearn.preprocessing import FunctionTransformer

# def preprocess_time(df):
#     df['created_at'] = pd.to_datetime(df['created_at'])
#     df['block_time'] = pd.to_datetime(df['block_time'])
#     df['created_year'] = df['created_at'].dt.year
#     df['created_month'] = df['created_at'].dt.month
#     df['created_day'] = df['created_at'].dt.day
#     df['created_hour'] = df['created_at'].dt.hour
#     df['created_day_sin'] = np.sin((2 * np.pi * df['created_day'] ) / 31)
#     df['created_day_cos'] = np.cos((2 * np.pi * df['created_day']) / 31)
#     df['created_hour_sin'] = np.sin((2 * np.pi * df['created_hour'])  / 24)
#     df['created_hour_cos'] = np.cos((2 * np.pi * df['created_hour']) / 24)
#     df['created_month_sin'] =  np.sin((2 * np.pi * df['created_month']) / 12)
#     df['created_month_cos'] = np.cos((2 * np.pi * df['created_month']) / 12)
#     df['created_at_unix'] = df['created_at'].astype(int) // 10**9  # Unix timestamp

#     df['block_year'] = df['block_time'].dt.year
#     df['block_month'] = df['block_time'].dt.month
#     df['block_day'] = df['block_time'].dt.day
#     df['block_hour'] = df['block_time'].dt.hour
#     df['block_day_sin'] = np.sin((2 * np.pi * df['block_day'] ) / 31)
#     df['block_day_cos'] = np.cos((2 * np.pi * df['block_day']) / 31)
#     df['block_hour_sin'] = np.sin((2 * np.pi * df['block_hour'])  / 24)
#     df['block_hour_cos'] = np.cos((2 * np.pi * df['block_hour']) / 24)
#     df['block_month_sin'] =  np.sin((2 * np.pi * df['block_month']) / 12)
#     df['block_month_cos'] = np.cos((2 * np.pi * df['block_month']) / 12)
#     df['block_time_unix'] = df['block_time'].astype(int) // 10**9  # Unix timestamp
#     return df.drop(columns=['created_at', 'block_time'])

# # def create_hash_encoding(df, hash_dict):
# #     for col, n_components in hash_dict.items():
# #         df[col] = df[col].astype('str')
# #         df[f'{col}_hash'] = df[col].apply(lambda x: hash(x) % n_components)
# #     return df.drop(columns=['name', 'symbol', 'token_uri'])

# # hash_dict = {
# #     'name':500,
# #     'symbol':300,
# #     'token_uri':1000,
# # }
# # preprocessor = ColumnTransformer(
# #     transformers = [
# #         ('datetime', FunctionTransformer(preprocess_time), ['created_at']),
# #         ('categorical', FunctionTransformer(create_hash_encoding, kw_args = {'hash_dict':hash_dict}), ['name', 'symbol', 'token_uri'])
        
# #     ],
# #     remainder='passthrough'
# # )
# # preprocessor.set_output(transform='pandas')

# from sklearn.base import TransformerMixin

# class BlockchainPreprocessor(TransformerMixin):
#     def __init__(self):
#         self.hash_dims = {
#             'name_x': 500,
#             'symbol_x': 200,
#             'token_uri': 800,
#             'creator': 1000,
#             'curve_address': 1500
#         }
#         self.numeric_clips = {}
#     def fit(self, X, y=None):
#         for col in ['slot_min', 'gas_used', 'dev_balance']:
#             q1 = X[col].quantile(0.25)
#             q3 = X[col].quantile(0.75)

#             self.numeric_clips[col] = (q1 - 1.5 * (q3 - q1), q3 + 1.5 * (q3 - q1))
#         return self

#     def transform(self, X):
#         df = X.copy()
#         df['coin_age'] = df['block_time_unix'] - df['created_at_unix']
#         df['peak_launch'] = df['created_hour'].between(9, 17).astype(int)
#         df = preprocess_time(df)
#         df['name'] = df['name_x'].fillna(df.get('name_y', ''))
#         df['symbol'] = df['symbol_x'].fillna(df.get('symbol_y', ''))
#         def parse_bundle_structure(s):
#             try:
#                 return {
#             'nft_actions': s.count('NFT_MINT'),
#             'swap_actions': s.count('TOKEN_SWAP'),
#             'other_actions': s.count('UNRECOGNIZED_ACTION')
#                     }
#             except:
#                 return {'nft_actions':0, 'swap_actions':0, 'other_actions':0}

#         bundle_data = df['bundle_structure'].apply(parse_bundle_structure)
#         df = pd.concat([df, bundle_data.apply(pd.Series)], axis=1)
        
#         # df['version'] = df['version'].fillna(0).astype(int)
        
#         for col, dim in self.hash_dims.items():
#             df[f'{col}_hash'] = df[col].astype(str).apply(lambda x: hash(x) % dim)
#         df['complexity_ratio'] = df['amount_of_instructions'] / (df['bundle_size'] + 1e-6)

#         for col in ['gas_used', 'dev_balance']:
#             df[col] = np.log1p(df[col].clip(lower=1))
#             df[col]  = df[col].clip(*self.numeric_clips[col])

#         for col in ['pf_program_index', 'creation_ix_index']:
#             freq = df[col].value_counts(normalize=True)
#             df[f'{col}_freq'] = df[col].map(freq)
#         to_drop = ['mint', 'slot_graduated', 'has_graduated',
#                   'name_x', 'symbol_x', 'token_uri', 'creator', 
#                   'curve_address', 'bundle_structure', 'url', 'version']
#         df = df.drop(columns=to_drop)
#         return df
    


# preprocessor = BlockchainPreprocessor()
# train_clean = preprocessor.fit_transform(train_merged_enh)
# test_clean = preprocessor.transform(test_merged_enh)
# # train_merged_2 = preprocessor.fit_transform(train_merged)
# # test_merged_2 = preprocessor.transform(test_merged)

In [40]:
# train_clean.to_csv('train_clean.csv', index=False)
# test_clean.to_csv('test_clean.csv', index=False)

In [41]:
cols_to_drop = ['created_at', 'block_time']
train_clean.drop(columns=cols_to_drop, inplace=True)
test_clean.drop(columns=cols_to_drop, inplace=True)

In [42]:
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import log_loss
from sklearn.base import clone
seed=1
def cv_score(X, y, model, folds=10, seed=seed):
    skf = StratifiedKFold(n_splits=folds, random_state=seed, shuffle=True)
    scores = []
    for i , (train_idx, valid_idx) in enumerate(skf.split(X, y), 1):
        X_train, X_valid = X.iloc[train_idx], X.iloc[valid_idx]
        y_train, y_valid = y.iloc[train_idx], y.iloc[valid_idx]

        clone_clf = clone(model)
        clone_clf.fit(X_train, y_train)
        preds = clone_clf.predict_proba(X_valid)[:,1]
        score = log_loss(y_valid, preds)
        scores.append(score)
        print(f'score at fold {i} :', score)

    print(f'Mean score across folds {i} :', np.mean(scores))

In [43]:
# from sklearn.preprocessing import StandardScaler
# scaler = StandardScaler()
# train_merged_scaled = scaler.fit_transform(train_merged_2[input_cols])
# test_merged_scaled = scaler.transform(test_merged_2[input_cols])

# train_merged_scaled_df = pd.DataFrame(train_merged_scaled, columns=input_cols)
# test_merged_scaled_df = pd.DataFrame(test_merged_scaled, columns=input_cols)

# train_merged_scaled_df = train_merged_scaled_df.fillna(0)
# test_merged_scaled_df = test_merged_scaled_df.fillna(0)

In [44]:
# problematic_chars = ['"', '\\', '[', ']', '{', '}', ':', ',']
# problematic_features = []

# for col in test_clean.columns:
#     if any(char in col for char in problematic_chars):
#         problematic_features.append(col)

# problematic_features

In [45]:
# train_clean = train_clean.rename(columns={'Unnamed: 0': 'unnamed'})
# test_clean = test_clean.rename(columns={'Unnamed: 0': 'unnamed'})

In [46]:
train_clean.to_csv('train_clean.csv', index=False)
test_clean.to_csv('test_clean.csv', index=False)

In [47]:
# import lightgbm as lgb
# import xgboost as xgb
# from sklearn.linear_model import LogisticRegression
# from sklearn.ensemble import ExtraTreesClassifier
# from sklearn.preprocessing import OrdinalEncoder
# # X = train_clean.drop(columns=['created_at', 'block_time'])
# X = train_clean.copy()
# # X = X.fillna(0)
# y = train['has_graduated'].copy()
# # value_counts = y.value_counts()
# # scale_weights = value_counts[0] / value_counts[1]
# # class_weights = np.where(y==1, 80, 20)
# cat_dtypes = X.select_dtypes(include=['object', 'category']).columns
# X[cat_dtypes] = X[cat_dtypes].fillna('missing').astype('category')
# # X[cat_dtypes] = X[cat_dtypes].astype('category')
# # numerical_dtypes = [cols for col in X.columns if col!='k']
# num_cols = X.select_dtypes(include=np.number).columns
# X[num_cols] = X[num_cols].fillna(0)
# ordinal_encoder = OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1)
# X[cat_dtypes] = ordinal_encoder.fit_transform(X[cat_dtypes])
# lgb_clf = LogisticRegression(random_state=1)
# # lgb_clf = xgb.XGBClassifier(objective='binary:logistic',
# #     eval_metric='logloss', random_state=1)
# # lgb_clf = lgb.LGBMClassifier(random_state=1)
# # lgb_clf = ExtraTreesClassifier(random_state=1, n_jobs=-1)
# # xgb_clf = xgb.XGBClassifier(random_state=1, n_estimators=6000, learning_rate=0.05, enable_categorical=True)
# lgb_clf.fit(X, y)
# # cv_score(X, y, lgb_clf)

In [48]:
# train_clean[cat_dtypes] = train_clean[cat_dtypes].fillna('missing').astype('category').copy()
# test_clean[cat_dtypes] = test_clean[cat_dtypes].fillna('missing').astype('category').copy()

In [49]:
# train

In [50]:
# # test_new = test['slot_min'].copy()
# test_clean[cat_dtypes] = test_clean[cat_dtypes].fillna('missing').astype('category')
# test_clean[cat_dtypes] = ordinal_encoder.transform(test_clean[cat_dtypes])
# test_clean[num_cols] = test_clean[num_cols].fillna(0)
# # test_clean[cat_dtypes] = ordinal_encoder.transform(test_clean[cat_dtypes])
# # test_clean = test_clean.fillna(0)
# preds = lgb_clf.predict_proba(test_clean)[:,1]
# preds

In [51]:
# test_clean.head(3)

In [52]:
# samp_sub['has_graduated'] = preds
# samp_sub.to_csv('submission_lgbm_clf.csv', index=False)

In [53]:
# import seaborn as sns
# sns.histplot(samp_sub['has_graduated'], bins=500, kde=True, color='r')
# # sns.histplot(y, bins=100, kde=True, color='purple')